In [ ]:
import ipyparallel
import socket
import os
import getpass
import holoviews as hv
import plot
from holoviews import opts
from holoviews import save
from holoviews import dim
import matplotlib.pyplot as plt
hv.extension('bokeh')
import adaptive
adaptive.notebook_extension()

import pickle 
import numpy as np
textwidth = 5.45776

In [ ]:
from matplotlib.patches import Rectangle

In [ ]:
plt.style.use(['default', './paper.mplstyle'])

In [ ]:
client = ipyparallel.Client(
    profile='slurm2104',  
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client[:].use_cloudpickle()

ids = client.ids
hosts = client[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
client2 = ipyparallel.Client(
    profile='slurm2104',  
    sshserver='iffslurm.iff.kfa-juelich.de'
)
client2[:].use_cloudpickle()

ids = client2.ids
hosts = client2[:].apply_sync(socket.gethostname)
print(ids)

notbookcwd = os.getcwd()
clustercwd = notbookcwd.replace('Home', 'Users/' + getpass.getuser())
print(len(client2[:].apply_sync(os.chdir, clustercwd)))

In [ ]:
param_list = ["Bi2Se3", "Bi2Se3_2", "Bi2Te3", "Sb2Te3", "Bi2Se3 Liu", "Bi2Te3 Liu", "Sb2Te3 Liu", 
              "Bi2Se3_8band", "Bi2Te3_8band", "Sb2Te3_8band", "toy_A","Bi2Se3 fit C.", "Bi2Te3 fit C.", "Sb2Te3 fit C.", 
              "Bi2Se3 fit", "Bi2Te3 fit", "Sb2Te3 fit",
              "Bi2Se3 Nach", "Bi2Te3 Nach", "Sb2Te3 Nach"]

In [ ]:
ham_type = ['3D (no $k^3$)', '2D', 'metal','4band','8band']

In [ ]:
def set_lattice_constant(params):
    try: 
        if len(params["p_mode"] > 0):
            try:
                index = np.where([a_optimal_4band[i]["p_mode"] == params["p_mode"][0] for i in np.arange(0,len(a_optimal_4band))])[0][0]
                params["a"]   = [ max(a_optimal_4band[index]["max_a_x"],a_optimal_4band[index]["max_a_y"]) ]
                params["a_z"] = [ a_optimal_4band[index]["max_a_z"] ]
            except:
                index = np.where([a_optimal_8band[i]["p_mode"] == params["p_mode"][0] for i in np.arange(0,len(a_optimal_8band))])[0][0]
                params["a"]   = [ max(a_optimal_4band[index]["max_a_x"],a_optimal_4band[index]["a_y"]) ]
                params["a_z"] = [ a_optimal_4band[index]["max_a_z"] ]
    except:
        try:
            index = np.where([a_optimal_4band[i]["p_mode"] == params["p_mode"] for i in np.arange(0,len(a_optimal_4band))])[0][0]
            params["a"]   = max(a_optimal_4band[index]["max_a_x"],a_optimal_4band[index]["max_a_y"])
            params["a_z"] = a_optimal_4band[index]["max_a_z"]
        except:
            index = np.where([a_optimal_8band[i]["p_mode"] == params["p_mode"] for i in np.arange(0,len(a_optimal_8band))])[0][0]
            params["a"]   = max(a_optimal_8band[index]["a_x"],a_optimal_8band[index]["a_y"])
            params["a_z"] = a_optimal_8band[index]["a_z"]
    return params

In [ ]:
import tools
import funcs
from operator import itemgetter
import numpy as np
import systems
import cmath
import pandas as pd

In [ ]:
def reload():
    import importlib
    import funcs
    import tools
    import systems
    
    importlib.reload(funcs)
    importlib.reload(tools)
    importlib.reload(systems)

reload()
_ = client[:].apply_sync(reload)

In [ ]:
_ = client2[:].apply_sync(reload)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,4))
indexes = [0,4,17,14,5,18,15,6,19,16]
labels = []

for j in np.arange(len(indexes)):
    i = indexes[j]
    try:
        A_0 = funcs.param_list[i]['A_0']
        B_0 = funcs.param_list[i]['B_0']
        M_0 = funcs.param_list[i]['M_0']
        C_1 = funcs.param_list[i]['C_1']
        M_1 = funcs.param_list[i]['M_1']
        C_2 = funcs.param_list[i]['C_2']
        M_2 = funcs.param_list[i]['M_2']
    except:
        A_0 = funcs.param_list[i]['A_perp']
        B_0 = funcs.param_list[i]['A_z']        
        M_0 = funcs.param_list[i]['M_0']
        C_1 = -funcs.param_list[i]['C_z']
        M_1 = -funcs.param_list[i]['M_z']
        C_2 = -funcs.param_list[i]['C_perp']
        M_2 = -funcs.param_list[i]['M_perp']
    labels.append(param_list[i])
    ax.plot(j, np.round(-M_0*(C_1/M_1),3),"kx",markersize=15,markeredgewidth=3)
    ax.plot(j, np.round(-M_0*(C_2/M_2),3),"k+",markersize=15,markeredgewidth=3)

#    print(param_list[i], 'z: ', np.round(-M_0*(C_1/M_1),3), 'x/y: ', 
#          np.round(-M_0*(C_2/M_2),3),'\t',abs(np.round(-M_0*(C_1/M_1)+M_0*(C_2/M_2),3)))
#    print(param_list[i], np.round(np.max([M_0+C_1*M_0/M_1, -M_0+C_1*M_0/M_1])/A_0,3), np.round(np.max([M_0+C_2*M_0/M_2, -M_0+C_2*M_0/M_2])/B_0,3))
#    print(param_list[i], np.round(np.max([M_0+C_1*M_0/M_1, -M_0+C_1*M_0/M_1])/(A_0*np.sqrt(1-C_1**2/M_1**2)),3), np.round(np.max([M_0+C_2*M_0/M_2, -M_0+C_2*M_0/M_2])/(B_0*np.sqrt(1-C_2**2/M_2**2)),3))
    q_m = .5*cmath.sqrt(B_0**2/(M_1**2-C_1**2)) - cmath.sqrt(.25*B_0**2/(M_1**2-C_1**2)+M_0/M_1)
    q_p = .5*cmath.sqrt(B_0**2/(M_1**2-C_1**2)) + cmath.sqrt(.25*B_0**2/(M_1**2-C_1**2)+M_0/M_1)
#    print(param_list[i],M_0,M_2,M_1,A_0,B_0,C_2,C_1)
    if i != 5:
        print(param_list[i],np.round(np.max([1/np.real(q_m),1/np.real(q_p)]),2))
    print('Delta E_DP',abs(np.round(-M_0*(C_1/M_1)+M_0*(C_2/M_2),3)))
    print('Distance from CB/VB',(abs(M_0)-M_0*(C_1/M_1))/abs(2*M_0),.5*(1-M_0/abs(M_0)*C_1/M_1))
    print()
ax.set_xticks(np.arange(len(indexes)));
ax.set_xticklabels(labels);
ax.set_ylabel("E [eV]");
ax.set_title("Position of Dirac point on the top/bottom (X) or side surface (+)")
plt.grid()

In [ ]:
-C_1/M_1

In [ ]:
len(funcs.param_list)

In [ ]:
funcs.param_list[16]

In [ ]:
delta_E = [0.061,0.043,0.445, 0.09,0.043,0.445, 0.09,0.055,0.313,0.059,0.151]
indexes = [0,1,2,3,10]
print('Max d')
for i in np.arange(len(indexes)):
    try:
        print(str(delta_E[i])+" & "+str( int(2*np.pi*funcs.param_list[indexes[i]]['B_0']/delta_E[i])))
    except:
        print(str(delta_E[i])+" & "+str( int(2*np.pi*funcs.param_list[indexes[i]]['A_z']/delta_E[i])))

#    print(sstring)     

In [ ]:
i = 10
N = 1000
k_range = [-.5,.5]
ks = np.linspace(*k_range,N)
E = funcs.get_continuum_spectrum(3, i, False, k_type='k_z', k_range=k_range, N=N, m_z=0)

fig,ax = plt.subplots(1,1,figsize=(4,4));
ax.plot(ks,E,"k.",lw=0, markersize=2, markeredgewidth=0);
ax.plot(-100,0,'k-', label='Continuum')

pparams = funcs.param_list[i]

C_1 = pparams['C_1']
C_2 = pparams['C_2']

M_0 = pparams['M_0']
M_1 = pparams['M_1']
M_2 = pparams['M_2']
    
ax.plot(ks, (C_1-M_1)*ks**2,'g--',label='$\hbar^2k_z^2/2m_{VB}$'); 
ax.plot(ks, (C_1+M_1)*ks**2,'b--',label='$\hbar^2k_z^2/2m_{CB}$');
ax.set_xlim(k_range)
ax.set_xlabel('$k_z~[1/\\AA]$'); ax.set_ylabel('E [eV]')
plt.legend();

In [ ]:
max_k = 0.3

In [ ]:
(0.26+0.26)*max_k**2, 2.75*max_k

## Weigth of CB and VB

In [ ]:
fig, ax = plt.subplots(6,2,figsize=(8,20),sharex=True,sharey=True)
indexes =  [11,12,13]
ks = np.linspace(-10,10,100)

for i in np.arange(len(indexes)):
    pparams = funcs.param_list[indexes[i]]
    
    C_1 = pparams['C_1']
    C_2 = pparams['C_2']
    
    M_0 = pparams['M_0']
    M_1 = pparams['M_1']
    M_2 = pparams['M_2']
    
    E_DP_z  = -M_0*(C_1/M_1)
    E_DP_xy = -M_0*(C_2/M_2)
    
    ax[i,0].plot(ks, (C_2-M_2)*ks**2,'g-'); ax[i,0].plot(ks, (C_2+M_2)*ks**2,'b-');
    ax[i,1].plot(ks, (C_1-M_1)*ks**2,'g-'); ax[i,1].plot(ks, (C_1+M_1)*ks**2,'b-');
    ax[i,1].set_title("$k_z$"); ax[i,0].set_title("$k_{x,y}$");

    CB_mass_xy = 1/(C_2+M_2)
    VB_mass_xy = -1/(C_2-M_2)
    
    CB_mass_z = 1/(C_1+M_1)
    VB_mass_z = -1/(C_1-M_1)
    
    print("VB mass (x/y): "+str(np.round(VB_mass_xy,3))+", CB mass (x/y): "+str(np.round(CB_mass_xy,3)))
    print("VB mass (z):   "+str(np.round(VB_mass_z,3))+", CB mass (z):   "+str(np.round(CB_mass_z,3)))    
    print(str(np.round( (VB_mass_xy-CB_mass_xy)/(VB_mass_xy+CB_mass_xy),3)),
          str(np.round( (VB_mass_z-CB_mass_z)/(VB_mass_z+CB_mass_z),3)))
    print("E_DP")
    print(str(np.round( -M_0*(VB_mass_xy-CB_mass_xy)/(VB_mass_xy+CB_mass_xy),3)),
          str(np.round( -M_0*(VB_mass_z-CB_mass_z)/(VB_mass_z+CB_mass_z),3))) 
    
    print("-------------------------------")

In [ ]:
import cmath

In [ ]:
i = 1
B_0 = funcs.param_list[i]['A_z']
M_0 = funcs.param_list[i]['M_0']
C_1 = -funcs.param_list[i]['C_z']
M_1 = -funcs.param_list[i]['M_z']
C_2 = -funcs.param_list[i]['C_perp']
M_2 = -funcs.param_list[i]['M_perp']

In [ ]:
i = 15
B_0 = funcs.param_list[i]['B_0']
M_0 = funcs.param_list[i]['M_0']
C_1 = funcs.param_list[i]['C_1']
M_1 = funcs.param_list[i]['M_1']
C_2 = funcs.param_list[i]['C_2']
M_2 = funcs.param_list[i]['M_2']

In [ ]:
B_0s=np.linspace(0,3,100)
q_m, q_p = [], []
for b_0 in B_0s:
    q_m.append( .5*cmath.sqrt(b_0**2/(M_1**2-C_1**2)) - cmath.sqrt(.25*b_0**2/(M_1**2-C_1**2)+M_0/M_1) )
    q_p.append( .5*cmath.sqrt(b_0**2/(M_1**2-C_1**2)) + cmath.sqrt(.25*b_0**2/(M_1**2-C_1**2)+M_0/M_1) )
    
lambda_z_inv = [min([q_m[i].real,q_p[i].real]) for i in range(len(q_m))]

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(3.5,1.75),layout='constrained')
fig.get_layout_engine().set(w_pad=0.03, h_pad=0, hspace=0, wspace=0)

#left, bottom, width, height = [0.6, 0.25, 0.35, 0.35]
#ax2 = fig.add_axes([left, bottom, width, height])

ax[0].plot(B_0s,lambda_z_inv,label='$\mathrm{min}\{\mathfrak{R}(q_z^\pm)\}$')
ax[0].plot(B_0s,np.array(q_p).imag,label='$\mathfrak{Im}(q_z^\pm)$')
#ax2.plot([2*np.sqrt(-M_0*(M_1**2-C_1**2)/M_1)]*2, [-1,1], 'k--')
ax[0].set_yticks([0,0.2,0.4]);#ax2.set_yticks([0,0.1,0.2])
ax[0].set_ylim(0,0.4)

ax[0].set_xticks(range(1+int(max(B_0s))))
ax[0].set_xlim(min(B_0s),max(B_0s))
ax[0].set_xlabel('$B_0~[\mathrm{eV\\AA}]$')
ax[0].legend()

d_cns = []
#ax[1].plot([B_0]*2, [10,110], color='k',ls='-')
for n in range(11):
    d_cns.append(n*np.pi/np.array(q_p).imag)
#    ax[1].plot(B_0s,d_cns[-1],'k-')
ax[1].fill_between(B_0s, d_cns[1], d_cns[2], color='gray', alpha=0.5,lw=0)
ax[1].fill_between(B_0s, d_cns[3], d_cns[4], color='gray', alpha=0.5,lw=0)
ax[1].fill_between(B_0s, d_cns[5], d_cns[6], color='gray', alpha=0.5,lw=0)
ax[1].fill_between(B_0s, d_cns[7], d_cns[8], color='gray', alpha=0.5,lw=0)
ax[1].fill_between(B_0s, d_cns[9], d_cns[10], color='gray', alpha=0.5,lw=0)

texts_y = np.array([25,25,35,48,60])
texts_x = np.array([2.2,1,.75,.45,.35])
texts = ['NI','QSH','NI','QSH','NI']
[ax[1].text(texts_x[i],texts_y[i],texts[i]) for i in range(len(texts))];
#
ax[1].set_xticks(range(1+int(max(B_0s))))
ax[1].set_xlim(min(B_0s),max(B_0s))
ax[1].set_ylim(20,70)
ax[1].set_yticks(np.arange(20,80,10),np.arange(2,8,1),minor=False)
ax[1].set_xlabel('$B_0~[\mathrm{eV\\AA}]$')
ax[1].set_ylabel('$d~[\mathrm{nm}]$')
ax[1].grid(axis='y'); ax[1].set_axisbelow(True); ax[1].minorticks_off();

caption = ['(a)','(b)']
shifts = [-0.3,-0.25]
for ii in range(len(caption)):
    ax[ii].text(
        shifts[ii], 1, 
        caption[ii],
        transform=ax[ii].transAxes
    )
#ax.legend()
plt.savefig('Im_q_d_cn_i'+str(i)+'.png')
plt.savefig('Im_q_d_cn_i'+str(i)+'.pdf',bbox_inches='tight')

In [ ]:
cols = 3
rows = 3
fontsize = 12

fig, ax = plt.subplots(
    rows, cols,
    figsize=(3*cols,2.5*rows),
    layout="constrained"
)
fig.get_layout_engine().set(h_pad=0.0, w_pad=0.0, wspace=0.1)

m_z_1 = 0
m_z_2 = m_z_1
N = 2000

k_vec = ["k_x","k_y","k_z"]
material = ["Bi$_2$Se$_3$","Bi$_2$Te$_3$","Sb$_2$Te$_3$"]
#k_ranges=[[0,.1],[0,0.05]]
k_ranges=[[0,.8]]*cols
#k_ranges=[[0,.1],[0,.1],[0,.05]]
E_ranges = [[-.5,.5],[-.6,.4],[-.6,.4]]
for j in np.arange(0,rows):
    for i in np.arange(0,cols):
        E = funcs.get_continuum_spectrum(0, 1+j, False, k_type=k_vec[i], k_range=k_ranges[i], N=N, m_z=m_z_1)
        ax[j,i].plot(np.linspace(*k_ranges[i],N),E,"k.",lw=0, markersize=2, markeredgewidth=0)
        E = funcs.get_continuum_spectrum(3, 4+j, False, k_type=k_vec[i], k_range=k_ranges[i], N=30, m_z=m_z_2)    
        ax[j,i].plot(np.linspace(*k_ranges[i],30),E,"r.",lw=0, markersize=10, markeredgewidth=0)
        ax[j,i].set_xlim(k_ranges[i])
        ax[j,i].set_ylim(E_ranges[j])
    ax[j,0].set_ylabel("E [eV]",fontsize=fontsize);


for i in np.arange(0,cols):
    ax[rows-1,i].set_xlabel("$"+k_vec[i]+"$ [$1/\AA$]",fontsize=fontsize); 
for j in np.arange(0,rows):
    ax[j,1].set_title(material[j],fontsize=fontsize+3);
ax[0,cols-1].plot([0],[-1],"k-",label="4 band")#, $M_z=$"+str(m_z_1))
#ax[0,cols-1].plot([0],[-1],"r-",label="4 band + $k^3$ term, $M_z=$"+str(m_z_2))
ax[0,cols-1].plot([0],[-1],"r-",label="4 band new")#, $M_z=$"+str(m_z_2))
ax[0,cols-1].legend();

#fname = "4_vs_8_band_M_z"+str(m_z_1)
fname = "4_band_M_z"+str(m_z_1)

directory = "continuum_spectra"
plt.savefig(fname+".png");
plt.savefig(directory+"/"+fname+".png");
fig.savefig(directory+"/"+fname+'.pdf',bbox_inches='tight')

In [ ]:
k_range=[0,0.4]
fig, ax = plt.subplots(1,2,figsize=(8,4))

p = funcs.param_list[10]
p['A_0'] = 3
p['B_0'] = 3
p['C_1'] = -5
Ex = funcs.get_continuum_spectrum(3, 10, False, params=p, k_type='k_x', k_range=k_range, N=1000, m_z=0)
Ez = funcs.get_continuum_spectrum(3, 10, False, params=p, k_type='k_z', k_range=k_range, N=1000, m_z=0)
ax[0].plot(np.linspace(*k_range,1000),Ex,"k.",lw=0, markersize=2, markeredgewidth=0)
ax[1].plot(np.linspace(*k_range,1000),Ez,"k.",lw=0, markersize=2, markeredgewidth=0)
ax[0].set_ylim(-2,2); ax[1].set_ylim(-2,2);

plt.show();

In [ ]:
def max_valid_a(a):
    
    k_type = "k_"+direction
    k_range = [-np.pi, np.pi]
        
    params = dict(
        a=1,
        a_z=1,
        L=1,
        W=1,
        T=1,
        delta=0,
        mu=0,
        h_mode=h_mode,
        m_z=0,
        ph_symmetry=False,
        num_bands=10,
        p_mode=p_mode,
        k_type=k_type, 
        directions="xyz",
        sym=False
    )    
    
    E_cont = funcs.get_continuum_spectrum(h_mode=h_mode, p_mode=p_mode, ph_symmetry=False, 
                             k_type=k_type, k_range=k_range, N=1000, m_z=0,params=params)
    
    upper_band_cont = np.max(E_cont,axis=1)
    lower_band_cont = np.min(E_cont,axis=1)
    
    gap_cont = min(upper_band_cont)-max(lower_band_cont)
     
    if direction == "z":
        params.update(a_z=a,T=a)
    else:
        params.update(a=a,W=a,L=a)
    
    Es = []
    for k in np.linspace(*k_range,1000):
        params.update(k = k*a)    
        Es.append(funcs.get_discretized_spectrum(**params)["Es"])
    Es = np.array(Es)
    
    upper_band_disc = np.max(Es,axis=1)
    lower_band_disc = np.min(Es,axis=1)    

    gap_disc = min(upper_band_disc)-max(lower_band_disc)    
    
    return gap_disc/gap_cont

In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
with open('a_optimal_90.pkl', 'rb') as f:
    a_optimal_90 = pickle.load(f)
with open('a_optimal_80.pkl', 'rb') as f:
    a_optimal_80 = pickle.load(f)
with open('a_optimal_4band_range.pkl', 'rb') as f:
    a_dict = pickle.load(f)    

In [ ]:
a_optimal_90, a_optimal_80

In [ ]:
fist_h_band_index = np.argmax(np.array([-1,-1,-12,-2]) > 0)
E_k0 = [1,2,3,4]
fist_h_band_E = E_k0[fist_h_band_index]
fist_h_band_E

In [ ]:
plt.plot(a_dict[-1]['a_x'],a_dict[-1]['gap_x'])

In [ ]:
h_mode = 3
p_mode = 15
gap_goal = 0.9
direction = "x"
learner = adaptive.Learner1D(max_valid_a,
                             bounds=(.1,60)
          )

In [ ]:
executor = client.executor()

In [ ]:
job = executor.submit(max_valid_a, a=42.52916666666667)

In [ ]:
res = job.result()

In [ ]:
max_valid_a( a=42.52916666666667 )

In [ ]:
syst = systems.make_lead(
         a=1, a_z=42.52916666666667, L=1, W=1, T=42.529167091958335,
         ph_symmetry=False,
         vector_potential="[0, - B_x * (z - {}), 0]".format(1),
         subst={'Delta': "Delta_lead(delta, x, y, z)"},
         ham_type='3D',
         directions='xyz'
     )

In [ ]:
get_discretized_spectrum(a, a_z, L, W, T, delta, m_z, mu, h_mode, p_mode, ph_symmetry, sym, num_bands, k, k_type, directions, **kwargs)

In [ ]:
runner.task.result()

In [ ]:
runner = adaptive.Runner(learner,
                           executor=executor,
                           goal=lambda l: l.loss() < 0.005
                          )
runner.live_info()

In [ ]:
a = learner.to_numpy()[:,0]
gap = learner.to_numpy()[:,1]

In [ ]:
if direction =="x":
    a_x   = a
    gap_x = gap
elif direction =="z":
    a_z   = a
    gap_z = gap

In [ ]:
a_dict.append(dict(h_mode=h_mode,p_mode=p_mode,a_x=a_x, gap_x=gap_x, a_y=a_x, gap_y=gap_x, a_z=a_z, gap_z=gap_z))

In [ ]:
len(a_dict)

In [ ]:
#fig, ax = plt.subplots(1,1,figsize=(10,6))
#plt.plot(a_dict[2]["a_z"],a_dict[2]["gap_z"]);
#ax.plot(a,gap);
#ax.set_ylim(0,1.5);

In [ ]:
[print(a_dict[i]['p_mode']) for i in np.arange(len(a_dict))]

In [ ]:
j = 3

In [ ]:
a_x   = a_dict[4]['a_x']
gap_x = a_dict[4]['gap_x']
a_z   = a_dict[4]['a_z']
gap_z = a_dict[4]['gap_z']

In [ ]:
for i in np.arange(len(a_x)):
    if gap_x[i]> 0.78 and gap_x[i]< 0.92:
        print(a_x[i],gap_x[i])

In [ ]:
for i in np.arange(len(a_z)):
    if gap_z[i]> 0.78 and gap_z[i]< 0.92:
        print(a_z[i],gap_z[i])

In [ ]:
a_optimal_802, a_optimal_902 = [], []

In [ ]:
a_optimal_80.append(dict(h_mode=a_dict[-1]['h_mode'], p_mode=a_dict[-1]['p_mode'], a=23.73, a_z=5.32 ))

In [ ]:
a_optimal_90.append(dict(h_mode=a_dict[-1]['h_mode'], p_mode=a_dict[-1]['p_mode'], a=19.13, a_z=3.84))

In [ ]:
a_optimal_80

In [ ]:
#a_optimal_80 = a_optimal_802 + a_optimal_80
a_optimal_90

In [ ]:
#with open('a_optimal_90.pkl', 'wb') as f:
#    pickle.dump(a_optimal_90, f)    

In [ ]:
#with open('a_optimal_4band_range.pkl', 'wb') as f:
#    pickle.dump(a_dict, f)

---

In [ ]:
with open('a_optimal_4band_range.pkl', 'rb') as f:
    a_dict = pickle.load(f)

fig, ax = plt.subplots(1,2,figsize=(10,4),layout='tight')
ax[0].plot([0,100,100,0],[0.8,0.8,0.9,0.9],"k:");
ax[0].plot(a_dict[1]["a_x"],a_dict[1]["gap_x"],label='Bi$_2$Se$_3$');
ax[0].plot(a_dict[2]["a_x"],a_dict[2]["gap_x"],label='Bi$_2$Te$_3$');
ax[0].plot(a_dict[3]["a_x"],a_dict[3]["gap_x"],label='Sb$_2$Te$_3$');

ax[0].set_xlim(0,35);
ax[0].set_ylim(0,1.3);
ax[0].set_xlabel('$a_{x,y}$ [$\AA$]');
ax[0].set_ylabel('Gap %');
ax[0].legend();
ax[0].grid();

ax[1].plot([0,100,100,0],[0.8,0.8,0.9,0.9],"k:");
ax[1].plot(a_dict[1]["a_z"],a_dict[1]["gap_z"],label='Bi$_2$Se$_3$');
ax[1].plot(a_dict[2]["a_z"],a_dict[2]["gap_z"],label='Bi$_2$Te$_3$');
ax[1].plot(a_dict[3]["a_z"],a_dict[3]["gap_z"],label='Sb$_2$Te$_3$');

ax[1].set_xlim(0,25);
ax[1].set_ylim(0,1.3);
ax[1].set_xlabel('$a_z$ [$\AA$]');
ax[1].legend();
ax[1].grid();

#plt.savefig('gap_vs_a_fit.png');
plt.savefig('gap_vs_a_4band.png');

In [ ]:
with open('a_optimal_4band_range.pkl', 'rb') as f:
    a_dict = pickle.load(f)

fig, ax = plt.subplots(1,2,figsize=(10,4),layout='tight')
ax[0].plot([0,100,100,0],[0.8,0.8,0.9,0.9],"k:");
ax[0].plot(a_dict[0]["a_x"],a_dict[0]["gap_x"],label='Bi$_2$Se$_3^{10}$');
ax[0].plot(a_dict[1]["a_x"],a_dict[1]["gap_x"],label='Bi$_2$Se$_3$');
ax[0].plot(a_dict[4]["a_x"],a_dict[4]["gap_x"],label='Toy model');

ax[0].set_xlim(0,35);
ax[0].set_ylim(0,1.3);
ax[0].set_xlabel('$a_{x,y}$ [$\AA$]');
ax[0].set_ylabel('Gap %');
ax[0].legend();
ax[0].grid();

ax[1].plot([0,100,100,0],[0.8,0.8,0.9,0.9],"k:");
ax[1].plot(a_dict[0]["a_z"],a_dict[0]["gap_z"],label='Bi$_2$Se$_3^{10}$');
ax[1].plot(a_dict[1]["a_z"],a_dict[1]["gap_z"],label='Bi$_2$Se$_3$');
ax[1].plot(a_dict[4]["a_z"],a_dict[4]["gap_z"],label='Toy model');

ax[1].set_xlim(0,25);
ax[1].set_ylim(0,1.3);
ax[1].set_xlabel('$a_z$ [$\AA$]');
ax[1].legend();
ax[1].grid();

plt.savefig('gap_vs_a_fit.png');

In [ ]:
params = dict(
    k=0,
    a=1,
    a_z=1,
    W=1,
    delta=0,
    flux=0,
    mu=0,
    m0=0,
    m1=0,
    D=0,
    h_mode=3,
    T=1,    
    m_z=0,
    ph_symmetry=0,
    num_bands=200,
    p_mode=0,
    sym=False,
    k_type="k_x",
    directions="xyz",
    C_0 = 0
)

executor = client.executor()

In [ ]:
params_disc = params.copy()
params_disc['directions'] = 'xyz'
if params_disc["h_mode"] == 0:
    rows = 4
    size = 2.5
else:
    rows = 3
    size = 3

directions = ['x','y','z']    
    
fig, ax = plt.subplots(
    rows,len(directions),
    figsize=(3*len(directions),size*rows),
    layout="constrained"
)

fig.get_layout_engine().set(h_pad=0.0, w_pad=0.0, hspace=0.0)

num = 50
fontsize = 12

colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
#colors = colors[3:]
for row in np.arange(0,rows):

    if params_disc["h_mode"] == 3:
        params_disc.update(p_mode=10+row)
    else:
        params_disc.update(p_mode=int(params_disc["h_mode"]/4*7)+row)

    if params_disc["h_mode"] == 0:
        a_s = [[8,15,20,30]]*rows
    else:
        a_s = [[5,10,15,20]]*rows
#    a_s = [[20,30,40,100]]*rows
#    a_s = [[1,2,4,10]]*rows
        
    k_ranges = [[0,1.05*np.pi/a_s[0][0]]]*rows   

    for j in np.arange(0,3):
        params_disc.update(k_type="k_"+directions[j])
        ks = np.linspace(*k_ranges[j],50)

        for i in np.arange(0,len(a_s[j])):
#            params_disc = set_lattice_constant(params_disc)       
            if directions[j] == 'z':
                params_disc.update(a_z = a_s[j][i], T = a_s[j][i], a=1, W=1, L=1)
            elif directions[j] == 'y':
                params_disc.update(a = a_s[j][i], W = a_s[j][i], a_z=1, T=1, L=a_s[j][i])
            else:
                params_disc.update(a = a_s[j][i], L = a_s[j][i], a_z=1, T=1, W=a_s[j][i])
            params_disc.update(T = params_disc["a_z"], W = params_disc["a"], L = params_disc["a"])
            Es = []
            for k in ks:
                params_disc.update(k = k*a_s[j][i])
                job = executor.submit(funcs.get_discretized_spectrum, **params_disc)
                Es.append(job.result()["Es"])
            Es = np.array(Es)
            ax[row][j].plot(ks[:-1], Es[:-1],marker='.', linewidth=0, color=colors[i])
            ax[row][j].plot(ks[-1]*np.ones(len(Es[-1])), Es[-1], label="a="+str(a_s[j][i])+" $\\AA$",marker='.', linewidth=0, color=colors[i])
            
            ax[row][j].plot([np.pi/a_s[j][i],np.pi/a_s[j][i]],[-1,1],"-", color=colors[i])
        if j == 1:
            ax[row][j].set_title("Ham. type="+str(ham_type[params_disc["h_mode"]])+", material="+str(param_list[params_disc["p_mode"]])+", translation inv. directions="+params_disc["directions"], fontsize=fontsize)#+
        ax[row][j].set_xlim(min(k_ranges[j]),max(k_ranges[j]))
        
        E = funcs.get_continuum_spectrum(h_mode=params_disc["h_mode"], p_mode=params_disc["p_mode"], ph_symmetry=False, 
                                     k_type=params_disc["k_type"], k_range=k_ranges[j], N=1000, m_z=params_disc["m_z"])
        ax[row][j].plot(np.linspace(min(ks),max(ks),1000),E,"k.",lw=0, markersize=2, markeredgewidth=0)

        ax[row][j].plot(-1,-1,"k-",label="Cont.")                
        
        ax[row][j].set_ylim(-1,1)
        
    ax[row][0].set_ylabel("E [eV]",fontsize=fontsize)
    
ax[rows-1][len(directions)-1].legend(loc='lower right');
plt.savefig("discretization/disc_effect_h_mode"+str(params_disc["h_mode"])+".png")#+"_p_mode"+str(params_disc["p_mode"])

In [ ]:
params = dict(
    k=0,
    a=1,
    a_z=-400,
    W=1,
    L=1,
    delta=0,
    flux=0,
    mu=-200,#42.839e-3 * np.array([0.77032, 1.3]), #np.linspace(1.75*42.83e-3, 2.25*42.83e-3, num=11),
    m0=0,
    m1=0,
    D=0,
    T=1,    
    m_z=0,
    ph_symmetry=0,
    num_bands=20,
    p_mode=0,
    h_mode=0,
    sym=False,
    k_type="k_x",
    directions="xy"
)

executor = client.executor()

In [ ]:
h_modes = [0,3, 3, 3,3, 3,3, 3, 3]
p_modes = [0,4,14,11,5,15,6,16,13]
ds = np.arange(20,80,10)
gaps = []

for i in range(len(p_modes)):
    gap = []
    for d in ds:
        
        params_disc = params.copy()
        params_disc.update(p_mode=p_modes[i],h_mode=h_modes[i],T=d)
        job = executor.submit(funcs.get_discretized_spectrum, **params_disc)
        Es = job.result()["Es"]
        
        above = Es[Es >  1e-10]        
        below = Es[Es < -1e-10]
        
        gap.append(min(above)-max(below))
    gaps.append(gap)
gaps = np.array(gaps)

In [ ]:
for ii in range(len(h_modes)):
    print(h_modes[ii], p_modes[ii])
    Str = ''
    for i in [int(i*1000) for i in gaps[ii]]:
        Str = Str+str(i)+ ' & '
    print(Str)

In [ ]:
params_hyb = dict(
    k=[0],
    a=[1],
    a_z=[-400],
    W=[1],
    L=[1],
    delta=[0],
    flux=[0],
    mu=[-200],
    m0=[0],
    m1=[0],
    D=[0],
    T=np.array([18,100]),    
    m_z=[0],
    ph_symmetry=[0],
    num_bands=[20],
    h_mode=[3],    
    p_mode=[0,4,6,17,18,19,14,15,16],
#    p_mode = [0,1,11],
#    p_mode=[2,12],
#    p_mode=[3,13],
    sym=[False],
    k_type=["k_x"],
    directions=["xy"]
)

In [ ]:
qa_hyb = tools.QuickAdptive(
    funcs.get_discretized_spectrum,
    learner_bounds_names=['T'],
    arg_picker=itemgetter('mn'),
    **params_hyb
)

qa_hyb.learner.strategy = "npoints"
qa_hyb.load('./data/band_structures')

In [ ]:
qa_hyb.runner.task.result()

In [ ]:
import adaptive
adaptive.notebook_extension()

qa_hyb.runn(
    goal=lambda learner: learner.npoints > 1000*len(params_hyb['p_mode']),
    executor=client
)

In [ ]:
qa_hyb.save('./data/band_structures')

In [ ]:
#qa_hyb.runner.learner.plot()

In [ ]:
#plt.plot(x[0],"x");

In [ ]:
import cmath 

x, Es = [], []
for pos in range(len(params_hyb['p_mode'])):
    x.append(list(qa_hyb.learner.learners[pos].extra_data.keys()))
    Es.append([qa_hyb.learner.learners[pos].extra_data[key]['Es'] for key in qa_hyb.learner.learners[pos].extra_data.keys()])

max_E = .3
fig, axs = plt.subplots(1,3,figsize=(7,2.5),layout = 'constrained',sharex=True,sharey=True)
fig.get_layout_engine().set(w_pad=0.03, h_pad=0.0, wspace=0.0, hspace=0.0)

lw = 1.5

colors = ['b','k','r']

if params_hyb['p_mode'] == [0,4,11,14,17]:
    labels = ['Bi$_2$Se$_3$ nat.','Bi$_2$Se$_3$ Zhang','Bi$_2$Se$_3$ fit C','Bi$_2$Se$_3$ fit P']
    experimental = [0.252,0.138,0.070,0.041]
elif params_hyb['p_mode'] == [5,12,15,18]:
    labels = ['Bi$_2$Te$_3$ Zhang','Bi$_2$Te$_3$ fit C','Bi$_2$Te$_3$ fit P']
    experimental = [0.14,0.036,0.011]
elif params_hyb['p_mode'] == [6,13,16,19]:
    labels = ['Sb$_2$Te$_3$ Zhang','Sb$_2$Te$_3$ fit C','Sb$_2$Te$_3$ fit P']
    experimental = [0.255, 0.060]
else:
    labels = ['Bi$_2$Se$_3$ [3]','Bi$_2$Se$_3$ [4]','Sb$_2$Te$_3$ [4]','Bi$_2$Se$_3$ [5]','Bi$_2$Te$_3$ [5]','Sb$_2$Te$_3$ [5]','Bi$_2$Se$_3$ Fit','Bi$_2$Te$_3$ Fit','Sb$_2$Te$_3$ Fit']#'Bi$_2$Se$_3$ Fit C','Bi$_2$Te$_3$ Fit C','Sb$_2$Te$_3$ Fit C',
    experimental = [[0.252,0.138,0.070,0.041,0],[0.14,0.036,0.011,0],[0.255, 0.060,0]]
    GW = [[0.238, 0.083, 0.034, 0.016, 0.007], [0.152, 0.026, 0.014, 0.003, 0.000], [0.254, 0.060, 0.004, 0.016, 0.012] ]
    GW_QSH = [ [0.152, 0.003, 0.000], [0.004, 0.016, 0.012] ]
    pos_QSH = [[20,50,60],[40,50,60]]
    colors = ['darkorchid','forestgreen','forestgreen','lightseagreen','lightseagreen','lightseagreen','darkorange','darkorange','darkorange','m','m','m']
if params_hyb['a_z'][0] == -20:
    shift = 1.08
elif params_hyb['a_z'][0] == -10:
    shift = 1.187
else:
    shift = 1

dds = np.arange(20,110,10)
gaps = []
xs = []
for ii in range(len(params_hyb['p_mode'])):
    if params_hyb['p_mode'][ii] in [0,1,4,11,14,17]:
        pi = 0
    elif params_hyb['p_mode'][ii] in [2,5,12,15,18]:
        pi = 1
    elif params_hyb['p_mode'][ii] in [3,6,13,16,19]:
        pi = 2
    pos = params_hyb['p_mode'][ii]
    if pos <= 3:
        B_0 = funcs.param_list[pos]['A_z']
        M_0 = funcs.param_list[pos]['M_0']
        C_1 = -funcs.param_list[pos]['C_z']
        M_1 = -funcs.param_list[pos]['M_z']          
    else:
        B_0 = funcs.param_list[pos]['B_0']
        M_0 = funcs.param_list[pos]['M_0'] 
        C_1 = funcs.param_list[pos]['C_1']
        M_1 = funcs.param_list[pos]['M_1']

    try:
        l_c = np.pi/(cmath.sqrt(.25*B_0**2/(M_1**2-C_1**2)+M_0/M_1)).imag
        print(pos,l_c)
    except:
        l_c = -1
    gap = []
    for i in range(len(Es[ii])):
        gap_tmp = min(Es[ii][i][Es[ii][i] >  1e-10]  )-max(Es[ii][i][Es[ii][i] < -1e-10])
        gap.append(gap_tmp)
    gap = np.array([y for _, y in sorted(zip(x[ii], gap))])
    
    x_sorted = np.sort(x[ii])*shift
    Label=labels[ii][12:]
    if colors[ii] != 'darkorange' and colors[ii] != 'm':
        Label = 'Ref. '+Label
    axs[pi].plot(x_sorted,gap,'-',markersize=lw,markeredgewidth=0,label=Label,color=colors[ii])
    [ axs[pi].fill_between(x_sorted    [ (n*l_c < x_sorted) & (x_sorted < (n+1)*l_c)],
                      np.zeros(len(gap))[ (n*l_c < x_sorted) & (x_sorted < (n+1)*l_c)],
                      gap               [ (n*l_c < x_sorted) & (x_sorted < (n+1)*l_c)],
                      color=colors[ii], alpha=.3,lw=0) for n in [1,3,5,7,9] ]
    gaps.append(gap)
    xs.append(x_sorted)
    axs[pi].set_ylim(0,max_E)
    axs[pi].set_xticks(dds,np.arange(2,11,1))
    axs[pi].set_xlim(min(dds)-2,max(dds))
    axs[pi].set_xlabel('$d~[\mathrm{nm}]$')
    if  14 < params_hyb['p_mode'][ii] < 17:
        axs[pi].plot(pos_QSH[params_hyb['p_mode'][ii]-15],GW_QSH[params_hyb['p_mode'][ii]-15],"o", markerfacecolor=(0,0,0, 1e-5), markeredgecolor='k',clip_on=False,zorder=100,ms=8)#,label=labels[params_hyb['p_mode'][ii]-11][:12]+' $GW$')

    if  13 < params_hyb['p_mode'][ii] < 17:
        axs[pi].plot((dds[:len(GW[params_hyb['p_mode'][ii]-14])]),GW[params_hyb['p_mode'][ii]-14],"kx",label='$GW$', clip_on=False,zorder=100)#,label=labels[params_hyb['p_mode'][ii]-11][:12]+' $GW$')
        axs[pi].plot((dds[:len(experimental[params_hyb['p_mode'][ii]-14])]),experimental[params_hyb['p_mode'][ii]-14],"r+",label='Exp.', clip_on=False,zorder=200)#,label=labels[params_hyb['p_mode'][ii]-11][:12]+' Exp.')
    
axs[0].set_yticks(np.arange(0,max_E+.05,.05), [int(i) for i in np.arange(0,max_E+.05,.05)*1e3] )   
axs[0].set_ylabel('$\Delta E_{\mathrm{hyb}}~[\mathrm{meV}]$');
axs[0].grid(axis='x'); axs[0].set_axisbelow(True)
axs[1].grid(axis='x'); axs[1].set_axisbelow(True)
axs[2].grid(axis='x'); axs[2].set_axisbelow(True)
axs[0].set_xlim(18,72);
axs[0].legend(loc='best',frameon=True,fontsize=8);

caption = ['(a)','(b)','(c)']
for i in range(len(caption)):
    if i == 0:
        sshift =-0.225
    else:
        sshift =-0.1
    axs[i].text(
        sshift, 1, 
        caption[i],
        transform=axs[i].transAxes
    )

plt.show();
fig.savefig('width_dependence/width_dependence_hyb_d_E_sym'+str(params_hyb['sym'][0])+'_p_mode'+str('_'.join([str(i) for i in params_hyb['p_mode']]))+'.png');      
fig.savefig('width_dependence/width_dependence_hyb_d_E_sym'+str(params_hyb['sym'][0])+'_p_mode'+str('_'.join([str(i) for i in params_hyb['p_mode']]))+'.pdf',bbox_inches='tight');      

In [ ]:
f_outname = "hybridization_gaps_sym"+str(bool(params_hyb['sym'][0]))+".pkl"

In [ ]:
with open(f_outname, 'wb') as f:
    pickle.dump([np.array(params_hyb['p_mode']), xs, gaps], f)

In [ ]:
with open('hybridization_gaps_symFalse.pkl', 'rb') as f:
    hybridization_gaps = pickle.load(f)

In [ ]:
hybridization_gaps[0]

In [ ]:
for i in range(len(hybridization_gaps[0])):
    print(params_hyb['p_mode'][i])
    for thickness in [20,30,40,50,60]:
        pos = np.where(np.array(hybridization_gaps[0]) == params_hyb['p_mode'][i])[0][0]
        pos_gap = np.where( (shift*hybridization_gaps[1][pos] < thickness*1.001) & (shift*hybridization_gaps[1][pos] > thickness*0.999))[0][0]
        print(hybridization_gaps[2][pos][pos_gap]*1e3)

In [ ]:
from matplotlib.patches import Rectangle

params = dict(
    k=0,
    mu=0,
    m_z=0,
    delta=0,
    ph_symmetry=False,
    sym=False,
    num_bands=150,
    k_type="k_x",
    directions="xy"
)

executor = client.executor()

surface_direction = ['z','y']
a_z = 1
a = 1
T = 200
W = 200
p_mode = 16
Dirs = ['xy','xz']
Label = ['Slab $x-y$, d='+str(T)+'$\\AA$', 'Slab $x-z$, d='+str(T)+'$\\AA$']   
directions = ['x','x']
    
num = 300
size = 1.5
ms = 1.5
ms2 = 1.5
lw = 1

params_disc = params.copy()
params_disc.update(h_mode = 3)
params_disc.update(p_mode = p_mode)

rows = 1

fig, ax = plt.subplots(
    1,2,
    figsize=(3.5,size+0.35),
    layout="constrained", sharex=True, sharey=True
)

fig.get_layout_engine().set(w_pad=0.03, h_pad=0.03, wspace=0.0, hspace=0.0)

    
colors = ['b', 'r','c',  'g', 'm', 'y', 'k']

k_range = [0,0.15]
p_modes = []
ks = np.linspace(*k_range,num)
Es_slab = []
for row in np.arange(0,rows):

    p_modes.append(str(p_mode))
    
    A_0 = funcs.param_list[p_mode]['A_0']
    B_0 = funcs.param_list[p_mode]['B_0']
    C_0 = funcs.param_list[p_mode]['C_0']    
    M_0 = funcs.param_list[p_mode]['M_0']
    C_1 = funcs.param_list[p_mode]['C_1']
    M_1 = funcs.param_list[p_mode]['M_1']
    C_2 = funcs.param_list[p_mode]['C_2']
    M_2 = funcs.param_list[p_mode]['M_2']
    
    pparams = funcs.param_list[p_mode]

    DP_z  = -M_0*(C_1/M_1)+C_0
    DP_xy = -M_0*(C_2/M_2)+C_0
    
    DPs = [DP_z, DP_xy]
    
    SS_1s = [ ks[:-1]*A_0*np.sqrt(1-C_1**2/M_1**2)-C_1*M_0/M_1+C_0, ks[:-1]*A_0*np.sqrt(1-C_2**2/M_2**2)-C_2*M_0/M_2+C_0 ]
    SS_2s = [-ks[:-1]*A_0*np.sqrt(1-C_1**2/M_1**2)-C_1*M_0/M_1+C_0,-ks[:-1]*A_0*np.sqrt(1-C_2**2/M_2**2)-C_2*M_0/M_2+C_0 ]
    
    for j in np.arange(0,len(directions)):
        
        params_disc.update(k_type="k_"+directions[j])
        if surface_direction[j] == 'z':
            params_disc.update(a_z=a_z, T=T,   a=a, W=a, L=a) #slab z
        else:
            params_disc.update(a_z=a_z, T=a_z, a=a, W=W, L=a) #slab y
        params_disc.update(directions=Dirs[j])
        Es = []
        for k in ks:
            if directions[j] == 'z':
                params_disc.update(k = k*a_z)
            else:
                params_disc.update(k = k*a)
            job = executor.submit(funcs.get_discretized_spectrum, **params_disc)
            Es.append(job.result()["Es"])
        Es = np.array(Es)
        Es_slab.append(Es)
        
        SS_u = [ np.min(Es[i][Es[i] > DPs[j]]) for i in range(len(Es)) ]
        SS_l = [ np.max(Es[i][Es[i] < DPs[j]]) for i in range(len(Es)) ]

        bulk_u = [ np.min(Es[i][Es[i] > SS_u[i]+1e-6]) for i in range(len(Es)) ]
        bulk_l = [ np.max(Es[i][Es[i] < SS_l[i]-1e-6]) for i in range(len(Es)) ]
        
        ax[j].fill_between(np.append(np.flip(-ks),ks), np.append(np.flip(bulk_u),bulk_u), 2, hatch='xxxx', edgecolor='k',facecolor='None')
        ax[j].fill_between(np.append(np.flip(-ks),ks), np.append(np.flip(bulk_l),bulk_l),-2, hatch='xxxx', edgecolor='k',facecolor='None')


        ax[j].plot( ks, SS_u, 'k.', markersize=ms, markeredgewidth=0,zorder=2)
        ax[j].plot(-ks, SS_u, 'k.', markersize=ms, markeredgewidth=0,zorder=2)
        ax[j].plot( ks, SS_l, 'k.', markersize=ms, markeredgewidth=0,zorder=2)
        ax[j].plot(-ks, SS_l, 'k.', markersize=ms, markeredgewidth=0,zorder=2)
        
        bulk = [ (Es[i] > bulk_u[i]) | (Es[i] < bulk_l[i]) for i in range(len(Es)) ]
        
#        ax[j].plot( np.array([ks]*40).T[bulk], Es[bulk], 'k.', lw=0, markersize=ms, markeredgewidth=0,zorder=1)
#        ax[j].plot(-np.array([ks]*40).T[bulk], Es[bulk], 'k.', lw=0, markersize=ms, markeredgewidth=0,zorder=1)
#        ax[j].plot( ks, Es, 'k.', lw=0, markersize=ms, markeredgewidth=0,zorder=2)
#        ax[j].plot(-ks, Es, 'k.', lw=0, markersize=ms, markeredgewidth=0,zorder=2)

        ax[j].plot([np.pi/params_disc['a_z'],np.pi/params_disc['a_z']],[-1,1],"--")
        
        ax[j].set_xticks([-.1,0,.1]);
        ax[j].set_xlim(-max(k_range),max(k_range))
    
        ax[j].plot([-2,2],[0,0],'-',color='gray',lw=.5,alpha=.75,zorder=0); 
        ax[j].plot([0,0],[-2,2],'-',color='gray',lw=.5,alpha=.75,zorder=0);
        ax[j].set_ylim(-.35,.35)
        ax[j].set_xlabel("$"+params_disc["k_type"]+"~[\mathrm{\\AA}^{-1}]$")
        
        ax[j].plot( ks[:-1], SS_1s[j],"--",color=colors[j],zorder=4)# + M_2*ks[:-1]**2,"k:")
        ax[j].plot(-ks[:-1], SS_1s[j],"--",color=colors[j],zorder=4)# + M_2*ks[:-1]**2,"k:")
        ax[j].plot( ks[:-1], SS_2s[j],"--",color=colors[j],zorder=4)# + M_2*ks[:-1]**2,"k:")
        ax[j].plot(-ks[:-1], SS_2s[j],"--",color=colors[j],zorder=4)# + M_2*ks[:-1]**2,"k:")

        ax[j].fill_between( ks[:-1], SS_1s[j], np.ones(len(SS_1s[j])), color=colors[j],alpha=0.5,lw=0,zorder=3)
        ax[j].fill_between(-ks[:-1], SS_1s[j], np.ones(len(SS_1s[j])), color=colors[j],alpha=0.5,lw=0,zorder=3)
        ax[j].fill_between( ks[:-1], SS_2s[j],-np.ones(len(SS_1s[j])), color=colors[j],alpha=0.5,lw=0,zorder=3)
        ax[j].fill_between(-ks[:-1], SS_2s[j],-np.ones(len(SS_1s[j])), color=colors[j],alpha=0.5,lw=0,zorder=3)
     
    # Pos. of DP, expected point were it joins the bulk
    #ax[0].plot([(min(0.2,abs(M_0))+abs(C_1*M_0/M_1))/A_0]*2,[-1,1],"g--")
    #ax[0].plot([(min(0.2,abs(M_0))-abs(C_1*M_0/M_1))/A_0]*2,[-1,1],"m--")
    
    #ax[1].plot([(min(0.2,abs(M_0))+abs(C_2*M_0/M_2))/A_0]*2,[-1,1],"g--")
    #ax[1].plot([(min(0.2,abs(M_0))-abs(C_2*M_0/M_2))/A_0]*2,[-1,1],"m--")
    
    #ax[0].plot([-1,1],[ - C_1*M_0/M_1]*2,"g:")
    #ax[1].plot([-1,1],[ - C_2*M_0/M_2]*2,"m:")
    
    ax[0].set_ylabel("$E~[\mathrm{eV}]$", labelpad=-4)

caption = ['(f)','(g)']#,'(c)','(d)','(e)','(f)','(g)','(h)']
for ii in range(len(caption)):
    if ii%2 == 0:
        sshift =-0.25
    else:
        sshift =-0.15
    ax[ii].text(
        sshift, 1, 
        caption[ii],
        transform=ax[ii].transAxes
    )
    
ax[ii].plot([-2,2],[0,0],'-',color='gray',lw=.5,alpha=.75,zorder=0); 
ax[ii].plot([0,0],[-2,2],'-',color='gray',lw=.5,alpha=.75,zorder=0);

plt.savefig("discretization/surface_h_mode"+str(params_disc["h_mode"])+"_p_mode"+str(''.join(p_modes))+"_a_z"+str(a_z)+"_T"+str(T)+"_a"+str(a)+"_W"+str(W)+".png")#+"_p_mode"+str(params_disc["p_mode"])
plt.savefig("discretization/surface_h_mode"+str(params_disc["h_mode"])+"_p_mode"+str(''.join(p_modes))+"_a_z"+str(a_z)+"_T"+str(T)+"_a"+str(a)+"_W"+str(W)+'.pdf',bbox_inches='tight')
plt.show();

In [ ]:
params = dict(
    k=0,
    a=1,
    a_z=1,
    W=1,
    delta=0,
    flux=0,
    mu=0,#42.839e-3 * np.array([0.77032, 1.3]), #np.linspace(1.75*42.83e-3, 2.25*42.83e-3, num=11),
    m0=0,
    m1=0,
    D=0,
    h_mode=0,
    T=1,    
#    m_z=np.linspace(0,0.01,0.001),
    m_z=0,
    ph_symmetry=0,
    num_bands=40,
    p_mode=0,
    k_type="k_x", 
    directions="xy"
)

executor = client.executor()

params_disc=params.copy()
k_range = [0,.3]
ks = np.linspace(*k_range,500)

a=1
W=a
L=a
T=50
a_z=T/100

params_disc.update(a=a, W=W, L=L, a_z=a_z, T=T)

E = funcs.get_continuum_spectrum(h_mode=params_disc["h_mode"], p_mode=params_disc["p_mode"], ph_symmetry=False, 
                             k_type=params_disc["k_type"], k_range=k_range, N=1000, m_z=params_disc["m_z"],params=params)
#ax.plot(np.linspace(min(ks),max(ks),1000),E,"k.",lw=0, markersize=2, markeredgewidth=0)
#ax.plot(-1,-1,"k-",label="cont.")
                            
Es = []
for k in ks:
    params_disc.update(k = k*params_disc['a']) 

    job = executor.submit(funcs.get_discretized_spectrum, **params_disc)
    Es.append(job.result()["Es"])
Es = np.array(Es)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

ax.scatter(np.repeat(ks,params_disc['num_bands']),Es, marker='.', linewidth=0)#, size=5, markeredgewidth=0)

ax.set_title(str(ham_type[params_disc["h_mode"]])+", "+str(param_list[params_disc["p_mode"]])+", T="+str(T))#, fontsize=10)#+        

ax.set_xlim(min(k_range),max(k_range))       
ax.set_ylim(-5,.5)

plt.savefig("discretization/nanoribbon_T_h_mode"+str(params_disc["h_mode"])+"_a_z"+str(a_z)+"_T"+str(T)+"_a"+str(a)+"_W"+str(W)+".png")#+"_p_mode"+str(params_disc["p_mode"])

# Test ribbon band structure

## Old params

In [ ]:
params = dict(
    k=[0,1],
    a=[10],
    a_z=[4],
    W=[500],
    delta=[0],
    flux=[0],
    mu=[0.25],#42.839e-3 * np.array([0.77032, 1.3]), #np.linspace(1.75*42.83e-3, 2.25*42.83e-3, num=11),
    D=[0],
    h_mode=[0],
    T=[30],    
#    m_z=np.linspace(0,0.01,0.001),
    m_z=[0],
    ph_symmetry=[0],
    num_bands=[100],
    m_eff=[1],
#    p_mode=[10],
    p_mode=[1],
    get_wf=[False],
    sym=[False],
    u_B=[0.1],
)

In [ ]:
# 3QL Bi2Se3
name='3QL_Bi2Se3'
params = dict(
    k=[-.06*10,.06*10],
    a=[10],
    a_z=[1],
    W=[500],
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[0],
    m0=[0.041],
    m1=[-15.82],
    D=[-13.94],
    v_F=[1.697],
    h_mode=[1],
    m_z=[0],
    ph_symmetry=[0],
    num_bands=[200],
    m_eff=[1],
    sym=[False],
    p_mode=[-1],
    get_wf=[True]
)

In [ ]:
# 5QL Sb2Te3
name='5QL_Sb2Te3'
params = dict(
    k=[-.06*4,.06*4],
    a=[2],
    a_z=[1],
    W=[1000],
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[0.03],
    m0=[-0.015],
    m1=[-13.10],
    D=[12.65],
    v_F=[2.870],
    h_mode=[1],
    m_z=[0],
    ph_symmetry=[0],
    num_bands=[100],
    m_eff=[1],
    sym=[False],
    p_mode=[-1],
    get_wf=[True]
)

In [ ]:
m0 = 0
m1 = 15
D  = 0
W = 500
a=1
v_F=3
params = dict(
    k=[-.02,.02],
    a=[a],
    a_z=[a],
    W=[W],#
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[0],# - m_z/2*D*m1/abs(D*m1) + 1e-3 ],# ],
    h_mode=[1],
#    m_z=[m_z],
    m_z=[0,0.04],#[0.05],#
    ph_symmetry=[0],
    num_bands=[10],
    m_eff=[1],
    sym=[True],
    p_mode=[-1],
    get_wf=[True],
    u_B=[0],
    u_T=[0]
)

## Determine relation of coinfinement energy and $m_0$

In [ ]:
m0  = [-0.017, 0.006]
m1  = [ 16.35, 13.83]
D   = [-13.31,-13.38]
v_F = [ 1.920, 2.887]
m_zs =[  0.02, 0.02]
mus = [0.0135, 0.001]

j = 0

qa = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    fname_prefix='data_learner_',
    learner_bounds_names=['k'],
    arg_picker=itemgetter('mn'),
    k=[-.2,.2],
    a=[10],
    a_z=[10],
    W=[500],
    T=[1],
    delta=[0],
    flux=[0],
    mu=[-(m0[j]+m_zs[j])*D[j]/m1[j]],
    h_mode=[1],
    S_imp=[0],
    m_z=[m_zs[j]],
    ph_symmetry=[0],
    num_bands=[30],
    m_eff=[1],
    sym=[False],
    p_mode=[-1],
    get_wf=[True],
    u_B=[0],
    u_T=[0],
    m0=[m0[j]],
    m1=[m1[j]],
    D=[D[j]],
    v_F=[v_F[j]]
)

In [ ]:
W = 500
a=4
params = dict(
    k=[0,0.003,0.006,0.009],
    a=[a],
    a_z=[a],
    W=[W],#
    T=[4],    
    delta=[0],
    flux=[0],
    mu=[0],# - m_z/2*D*m1/abs(D*m1) + 1e-3 ],# ],
    h_mode=[1],
    S_imp=[0],    
#    m_z=[m_z],
    m_z=[0,0.04],#[0.05],#
    ph_symmetry=[False],
    num_bands=[30],
    m_eff=[1],
    sym=[False],
    p_mode=[-1],
    get_wf=[True],
    u_B=[0],
    u_T=[0],
)

In [ ]:
W = 500
a=4
params = dict(
    k=[-.04,.04],
    a=[a],
    a_z=[a],
    W=[W],#
    T=[4],    
    delta=[0],
    flux=[0],
    mu=[0],# - m_z/2*D*m1/abs(D*m1) + 1e-3 ],# ],
    h_mode=[1],
    S_imp=[0],    
#    m_z=[m_z],
    m_z=[0.02],#[0.05],#
    ph_symmetry=[True],
    num_bands=[30],
    m_eff=[1],
    sym=[False],
    p_mode=[-1],
    get_wf=[True],
    u_B=[0],
    u_T=[0],
)

In [ ]:
qa = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
#    learner_bounds_names=['k'],
#    learner_bounds_names=['W'],
    learner_bounds_names=['k'],
    arg_picker=itemgetter('mn'),
    **params
)

qa.learner.strategy = "npoints"

## 2nd runner

In [ ]:
params2 = params.copy()
#params2['mu'][0] = params2['mu'][0]+0.008
#params2['delta'][0] = 0.001
#params2['ph_symmetry'][0] = True
params2['m0']=[-.05,.05]
params2['m_z']=[0]
qa2 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m0'],
#    learner_bounds_names=['k'],    
#    loss_per_interval=[uniform_loss],
    arg_picker=itemgetter('Es'),
    **params2
)

qa2.learner.strategy = "npoints"

In [ ]:
qa.load('./data/band_structures')
qa2.load('./data/band_structures')

In [ ]:
import adaptive
adaptive.notebook_extension()

qa.runn(
    goal=lambda learner: learner.npoints > 400,#799
    executor=client
)

In [ ]:
qa.runner.live_info()

In [ ]:
#qa.runner.task.result()

In [ ]:
fig, ax =plt.subplots(1,1,figsize=(4,4))#,sharey=True,sharex=True)

ks = qa.learner.learners[0].to_dataframe()['x'].values
Es = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))]
wfs = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(ks))]

colors = ['b', 'g', 'r', 'c', 'm']

for i in range(len(Es)):
    ax.plot(ks[i]*np.ones(len(Es[i])), Es[i],'k.',ms=1);

ax.set_ylim(-.05,.05)
ax.plot([-.2,.2],[0,0],"k--")

In [ ]:
import adaptive
adaptive.notebook_extension()

qa2.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)

In [ ]:
qa2.runner.live_info()

In [ ]:
qa.save('./data/band_structures')
qa2.save('./data/band_structures')

## Band structure

In [ ]:
import kwant

In [ ]:
fig, ax =plt.subplots(4,4,figsize=(12,8))

#which_bands=np.array([[15,15,15],[16,16,16]])
which_bands=np.array([16,16,16])#4QL SbTe 500
#which_bands=np.array([[16,15,15],[17,17,17]])#4QL SbTe 1000
#which_bands=np.array([[17,16,15],[18,17,17]])
#which_bands=np.array([[17,15,15],[18,17,17]])
#which_bands=np.array([[47,45,67],[48,47,69]])#5QL BiTe
#which_bands=np.array([[46,56,72],[47,58,74]])#6QL BiTe
colors = ['b', 'g', 'r', 'c', 'm']
poss= [10,35,70]

for j in [0,1,2,3]:

    for ii in [0,j]:
        x = qa.learner.learners[ii].to_dataframe()['x'].values
        Es = [qa.learner.learners[ii].to_dataframe()['extra_data'][i]['Es'] for i in range(len(x))]
        wfs = [qa.learner.learners[ii].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(x))]
    
        #ks2 = qa2.learner.learners[0].to_dataframe()['x'].values
        #Es2 = [qa2.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks2))]
           
        for i in range(len(Es)):
            ax[j,0].plot(x[i]*np.ones(len(Es[i])), Es[i],['k.','r.','r.','r.'][ii],ms=1);
        
    for i in range(len(poss)):
        pos = poss[i]
        ax[j,0].plot(x[pos], Es[pos][which_bands[i]], colors[i]+"x",ms=10);
 
    lead = systems.make_lead(
        a=params['a'][0], a_z=params['a'][0], W=params['W'][0], T=params['T'][0],
        ph_symmetry=False,
        vector_potential="[0, - B_x * (z - {}), 0]".format(0),
        subst={'Delta': "Delta_lead(delta, x, y, z)",
               'U':   "U_lead(u_B, u_T, x, y, z)"},
        ham_type='2D'
    )
    
    rho = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_0, systems.sigma_0))
    rhoz = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_z, systems.sigma_0))

    densitys,densitysz = [], []
    for i in range(len(poss)):
        densitys.append(  rho(wfs[poss[i]][:,which_bands[i]]) )
        densitysz.append( rhoz(wfs[poss[i]][:,which_bands[i]]) )

    for i in range(len(poss)):
        ax[j,i+1].plot(np.arange(len(densitys[i])),densitys[i],"k-")
        ax[j,i+1].plot(np.arange(len(densitysz[i])),densitysz[i],"b-")
        ax[j,i+1].set_ylim(-.05,0.05);
    
    for i in range(4):
        ax[j,i].grid();
    
    ax[j,0].set_xlim(params['m_z']); ax[j,0].set_ylim(0.01,.04);
    ax[j,0].plot([0.018547416]*2,[-1,1],"r-");

plt.savefig("low_m0_asym_p_mode"+str(params['p_mode'][0])+"_T"+str(params['T'][0])+".png");

In [ ]:
fig, ax =plt.subplots(1,2,figsize=(8,4))

which_bands=[17]*7+[16]*9+[16]*84
colors = ['b', 'g', 'r', 'c', 'm']
poss= np.arange(100)

for j in [0]:

    x = qa.learner.learners[0].to_dataframe()['x'].values
    Es = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(x))]
    wfs = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(x))]
        
    for i in range(len(poss)):
        pos = poss[i]
        ax[0].plot(x[pos], Es[pos][which_bands[i]], "bo",ms=4);
        
    for i in range(len(Es)):
        ax[0].plot(x[i]*np.ones(len(Es[i])), Es[i],'k.',ms=1);

 
    lead = systems.make_lead(
        a=params['a'][0], a_z=params['a'][0], W=params['W'][0], T=params['T'][0],
        ph_symmetry=False,
        vector_potential="[0, - B_x * (z - {}), 0]".format(0),
        subst={'Delta': "Delta_lead(delta, x, y, z)",
               'U':   "U_lead(u_B, u_T, x, y, z)"},
        ham_type='2D'
    )
    
    rho = kwant.operator.Density(lead.finalized(), np.kron( systems.sigma_0, systems.sigma_0))
    rhoz = kwant.operator.Density(lead.finalized(), np.kron(  systems.sigma_z, systems.sigma_0))

    densitys,densitysz = [], []
    for i in range(len(poss)):
        densitys.append(  rho(wfs[poss[i]][:,which_bands[i]]) )
        densitysz.append( np.sum(rhoz(wfs[poss[i]][:,which_bands[i]]) ))

    ax[1].plot(x[poss],densitysz)

    #for i in range(len(poss)):
    #    ax[j,i+1].plot(np.arange(len(densitys[i])),densitys[i],"k-")
    #    ax[j,i+1].plot(np.arange(len(densitysz[i])),densitysz[i],"b-")
    #    ax[j,i+1].set_ylim(-.05,0.05);
    
    #ax[j,0].plot([np.pi*params['v_F'][0]/250]*2,[-1,1],"b--")
    #ax[j,0].plot([params['m0'][0]]*2,[-1,1],"b:")
    
    ax[0].set_xlim(0,0.03); ax[0].set_ylim(0.0,.07); 
    ax[0].set_xlabel("Mz [eV]"); ax[0].set_ylabel("E [eV]");
    ax[1].set_xlabel("Mz [eV]"); ax[1].set_ylabel("Spin. pol.");
    ax[0].plot([0.018547416]*2,[-1,1],"r-");    ax[1].plot([0.018547416]*2,[-1,1],"r-");   
#plt.savefig("low_m0_asym_p_mode"+str(params['p_mode'][0])+"_T"+str(params['T'][0])+".png");

In [ ]:
fig, ax =plt.subplots(1,2,figsize=(12,8))

which_bands=[17]*190
which_bands2=[12]*190
colors = ['b', 'g', 'r', 'c', 'm']
poss= np.arange(190)
wfs= []

for j in [0,1,2,3]:
    x = qa.learner.learners[j].to_dataframe()['x'].values
    wfs.append( [qa.learner.learners[j].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(x))] )

for j in [0,1]:

    x = qa.learner.learners[j].to_dataframe()['x'].values
    Es = [qa.learner.learners[j].to_dataframe()['extra_data'][i]['Es'] for i in range(len(x))]

    #ks2 = qa2.learner.learners[0].to_dataframe()['x'].values
    #Es2 = [qa2.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks2))]
       
    for i in range(len(poss)):
        pos = poss[i]
        ax[0].plot(x[pos], Es[pos][which_bands[i]], ['ko','bo','bo'][j],ms=4);
        ax[0].plot(x[pos], Es[pos][which_bands2[i]], ['ko','bo','bo'][j],ms=4);
        
    for i in range(len(Es)):
        ax[0].plot(x[i]*np.ones(len(Es[i])), Es[i],['k.','r.','r.'][j],ms=1);
        
    lead = systems.make_lead(
        a=params['a'][0], a_z=params['a'][0], W=params['W'][0], T=params['T'][0],
        ph_symmetry=True,
        vector_potential="[0, - B_x * (z - {}), 0]".format(0),
        subst={'Delta': "Delta_lead(delta, x, y, z)",
               'U':   "U_lead(u_B, u_T, x, y, z)"},
        ham_type='2D'
    )

    rho = kwant.operator.Density(lead.finalized(), np.kron( np.kron(systems.sigma_0, systems.sigma_0), systems.sigma_0))
    rhoz = kwant.operator.Density(lead.finalized(), np.kron( np.kron(systems.sigma_0, systems.sigma_z), systems.sigma_0))

    densitys,densitysz = [], []
    for i in range(len(poss)):
        densitys.append(  rho(wfs[j][poss[i]][:,which_bands[i]]) )
        densitysz.append( np.sum(rhoz(wfs[j][poss[i]][:,which_bands[i]]) ))

    ax[1].plot(x[poss],densitysz,'.')

    ax[0].set_ylim(-0.04,0.04);


for j in [1,2,3]:
    mat_elem = []
    mat_elem2 = []
    for i in range(len(poss)):
        wf2 = np.array([np.matmul(np.kron(np.array([[0,1],[0,0]]),np.kron(np.array([[0,1],[-1,0]]),np.array([[1,0],[0,0]]))),np.reshape(wfs[j][poss[i]][:,which_bands2[i]],(126,8))[ii]) for ii in range(126)]).flatten()
        mat_elem.append(  np.dot(wfs[j][poss[i]][:,which_bands[i]].conj(),wf2 ))
        mat_elem2.append( np.dot(wfs[j][poss[i]][:,which_bands[i]].conj(),wfs[j][poss[i]][:,which_bands2[i]] ))
    mat_elem = np.array(mat_elem)
    mat_elem2 = np.array(mat_elem2)
    ax[1].plot(x[poss],abs(mat_elem),'.',label='k_x='+['.0025','.005','.0075'][j-1])
ax[1].legend();
#ax[1].plot(x[poss],abs(mat_elem2),'g.')
#plt.savefig("low_m0_asym_p_mode"+str(params['p_mode'][0])+"_T"+str(params['T'][0])+".png");

In [ ]:
fig, ax =plt.subplots(1,7,figsize=(15,6))

which_bands=[17]*400
which_bands2=[12]*400
colors = ['b', 'g', 'r', 'c', 'm']
poss= np.arange(400)

x = qa.learner.learners[0].to_dataframe()['x'].values
wfs = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(x))]

x = qa.learner.learners[0].to_dataframe()['x'].values
Es = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(x))]
   
for i in range(len(poss)):
    pos = poss[i]
    ax[0].plot(x[pos], Es[pos][which_bands[i]], 'ro',ms=4);
    ax[0].plot(x[pos], Es[pos][which_bands2[i]], 'ro',ms=4);
    
for i in range(len(Es)):
    ax[0].plot(x[i]*np.ones(len(Es[i])), Es[i],'k.',ms=1);
    
lead = systems.make_lead(
    a=params['a'][0], a_z=params['a'][0], W=params['W'][0], T=params['T'][0],
    ph_symmetry=True,
    vector_potential="[0, - B_x * (z - {}), 0]".format(0),
    subst={'Delta': "Delta_lead(delta, x, y, z)",
           'U':   "U_lead(u_B, u_T, x, y, z)"},
    ham_type='2D'
)

rho = kwant.operator.Density(lead.finalized(), np.kron( np.kron(systems.sigma_0, systems.sigma_0), systems.sigma_0))
rhoxup   = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, .5*np.array([[1,1],[1,1]])), systems.sigma_0))
rhoxdown = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, .5*np.array([[1,-1],[-1,1]])), systems.sigma_0))
rhoyup   = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, .5*np.array([[1,-1j],[1j,1]])), systems.sigma_0))
rhoydown = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, .5*np.array([[1,1j],[-1j,1]])), systems.sigma_0))
rhozup   = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, np.array([[1,0],[0,0]])), systems.sigma_0))
rhozdown = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, np.array([[0,0],[0,1]])), systems.sigma_0))

rhotop   = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0, systems.sigma_0), np.array([[1,0],[0,0]])))
rhobottom = kwant.operator.Density(lead.finalized(), np.kron( np.kron( systems.sigma_0,systems.sigma_0), np.array([[0,0],[0,1]])))

densitys = []
densitysxup,densitysxdown = [], []
densitysyup,densitysydown = [], []
densityszup,densityszdown = [], []
densitystop,densitysbottom = [], []
for i in range(len(poss)):
    densitys.append(  rho(wfs[poss[i]][:,which_bands[i]]) )
    densitysxup.append( np.sum(rhoxup(wfs[poss[i]][:,which_bands[i]]) ))
    densitysxdown.append( np.sum(rhoxdown(wfs[poss[i]][:,which_bands[i]]) ))
    densitysyup.append( np.sum(rhoyup(wfs[poss[i]][:,which_bands[i]]) ))
    densitysydown.append( np.sum(rhoydown(wfs[poss[i]][:,which_bands[i]]) ))
    densityszup.append( np.sum(rhozup(wfs[poss[i]][:,which_bands[i]]) ))
    densityszdown.append( np.sum(rhozdown(wfs[poss[i]][:,which_bands[i]]) ))
    densitystop.append( np.sum(rhotop(wfs[poss[i]][:,which_bands[i]]) ))
    densitysbottom.append( np.sum(rhobottom(wfs[poss[i]][:,which_bands[i]]) ))
    
ax[2].plot(x[poss],densitysxup,'s',label="spin x up")
ax[2].plot(x[poss],densitysxdown,'s',label="spin x down")
ax[3].plot(x[poss],densitysyup,'d',label="spin y up")
ax[3].plot(x[poss],densitysydown,'d',label="spin y down")
ax[4].plot(x[poss],densityszup,'X',label="spin z up")
ax[4].plot(x[poss],densityszdown,'X',label="spin z down")
ax[5].plot(x[poss],densitystop,'o',label="top")
ax[5].plot(x[poss],densitysbottom,'o',label="bottom")

for i in range(len(densitys)):
    if(x[i]>0):
        ax[6].plot(np.arange(126),densitys[i]+10*x[i],'k-')
    
mat_elem = []
mat_elem2 = []
for i in range(len(poss)):
    wf2 = np.array([np.matmul(np.kron(np.array([[0,1],[0,0]]),np.kron(np.array([[0,1],[-1,0]]),np.array([[1,0],[0,0]]))),np.reshape(wfs[poss[i]][:,which_bands2[i]],(126,8))[ii]) for ii in range(126)]).flatten()
    mat_elem.append(  np.dot(wfs[poss[i]][:,which_bands[i]].conj(),wf2 ))
mat_elem = np.array(mat_elem)
mat_elem2 = np.array(mat_elem2)
ax[1].plot(x[poss],abs(mat_elem),'-',label="ind. gap")
[ax[i].legend() for i in np.arange(1,6)]
[ax[i].set_ylim(0,1) for i in np.arange(1,6)]
ax[0].set_ylim(-0.05,0.05);
#ax[1].plot(x[poss],abs(mat_elem2),'g.')
#plt.savefig("low_m0_asym_p_mode"+str(params['p_mode'][0])+"_T"+str(params['T'][0])+".png");

In [ ]:
np.reshape(wfs[1][poss[i]][:,which_bands[i]],(126,4))[0]

In [ ]:
len(np.reshape(wfs[1][poss[i]][:,which_bands[i]],(126,4)))

In [ ]:
import kwant

In [ ]:
fig, ax =plt.subplots(1,2,figsize=(8,4))#,sharey=True,sharex=True)

ks = qa.learner.learners[0].to_dataframe()['x'].values
Es = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))]
wfs = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['wfs'] for i in range(len(ks))]

colors = ['b', 'g', 'r', 'c', 'm']

for i in range(len(Es)):
    ax[0].plot(ks[i]*np.ones(len(Es[i])), Es[i],'k.',ms=1);
#for i in range(len(Es2)):
#    plt.plot(ks2[i]*np.ones(len(Es2[i])), Es2[i],'r.',ms=5);   

ax[0].plot(ks[pos1[0]], Es[pos1[0]][pos1[1]],"ro", ms=10)
ax[0].plot(ks[pos2[0]], Es[pos2[0]][pos2[1]],"bo", ms=10)

#plt.ylim(-.01,.01); plt.xlim(-.017,.017);
ax[0].set_ylim(-.15,.15); ax[0].set_xlim(-.2,.2)
ax[0].set_xlabel("$k$"); ax[0].set_ylabel("$E$ [eV]");
ax[0].grid();

ax[1].plot(np.arange(0,500/params['a'][0]+1),density1,"r-")
print(sum(density1))

exponential = np.exp(-np.linspace(0,params['W'][0],100)*(-params['m0'][0]+params['m_z'][0])/params['v_F'][0])
ax[1].plot(np.linspace(0,params['W'][0]/params['a'][0],100),exponential/(np.sum(exponential)),"b--")

ax[1].plot(np.arange(0,500/params['a'][0]+1),density2,"b-")
print(sum(density2))

ax[1].set_xticks(np.arange(0,params['W'][0]/params['a'][0]+1,100/params['a'][0]),np.arange(0,params['W'][0]/10+1,10))
ax[1].set_xlabel("$y$ [nm]"); ax[1].set_ylabel("$|\Psi|^2$");

#plt.title(str(param_list[p_mode])+", T="+str(int(params['T'][0]/10))+" QL, $u_B$="+str(params['u_B'][0])+" eV, $u_T$="+str(params['u_T'][0])+" eV");
fig.tight_layout()
plt.savefig("band_structure_p_mode"+str(p_mode)+"_T"+str(int(params['T'][0]/10))+"_u_B"+str(params['u_B'][0])+"_u_T"+str(params['u_T'][0])+".png")

In [ ]:
pparams = funcs.param_list[15]
pparams.update(u_B=0.1)
pparams.update(funcs.default_params)
pparams.update(Delta_lead = funcs.delta_profile_funtion(1))
pparams.update(U_B_lead = funcs.u_profile_funtion(1))

funcs.calculate_majorana_num(a, a_z, T, W, params2['mu'][0], m_z, 0.001, 3, pparams)

In [ ]:
ks2 = qa2.learner.learners[0].to_dataframe()['x'].values
Es2 = [qa2.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks2))]

ks = qa.learner.learners[0].to_dataframe()['x'].values
Es = [qa.learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))]
colors = ['b', 'g', 'r', 'c', 'm']

if abs(params['m_z'][0]) > abs(params['m0'][0]) or params['m0'][0]*params['m1'][0] > 0:
    mu = -(params['m0'][0]-params['m_z'][0])*params['D'][0]/params['m1'][0] #for large magnetization the interesting physics will be around the chiral edge mode
else:
    mu = 0
    
for i in range(len(Es2)):
    plt.plot(ks2[i]*np.ones(len(Es2[i])), Es2[i],'kx',ms=1);
for i in range(len(Es2)):
    plt.plot(-ks2[i]*np.ones(len(Es2[i])), Es2[i],'kx',ms=1);
for i in range(len(Es)):
    plt.plot(ks[i]*np.ones(len(Es[i])), Es[i],'r.',ms=1);
for i in range(len(Es)):
    plt.plot(-ks[i]*np.ones(len(Es[i])), Es[i],'r.',ms=1);    

#plt.ylim(-.01,.01); plt.xlim(-.017,.017);
plt.ylim(-.2,.2); plt.xlim(0,7)
plt.grid();

#plt.plot([-1,1],[-params['mu'][0]-(params['D'][0]*params['m1'][0])/abs((params['D'][0]*params['m1'][0]))*abs(params['m_z'][0]-params['m0'][0]) +(params['m_z'][0]-params['m0'][0])*params['D'][0]/(params['m1'][0])]*2,"b--")

#with open(name+'_thin-film_ribbon.pkl', 'wb') as f:
#    pickle.dump([ks,Es], f)

#plt.savefig("spectrum_large_avg_gap_mu="+str(params['mu'][0])+".png")

## Create w.f. visualization

In [ ]:
import kwant

In [ ]:
executor=client.executor()

In [ ]:
#m0s = np.array([[0.05,-0.05]]*5).T
m0s = np.array([[0.05,0.01,-0.01,-0.05]]*3).T
m1 = 15
D  = 0
#m_zs = np.array([[0,0.03,0.04,0.05,0.06]]*2)
m_zs = np.array([[1e-6,0.045,0.065]]*4)
W = 500
a=1
v_F=3

jobs = []
qas = []
for i in range(len(m0s)):
    jobs_tmp = []
    qa_tmps = []
    for j in range(len(m0s[i])):
        jobs_tmp.append(executor.submit(funcs.calculate_lead_spectrum,k=0, a=a, a_z=1,W=500,T=0,delta=0,flux=0,mu=0,m0=m0s[i,j],m1=m1,D=D,v_F=v_F,h_mode=1,m_z=m_zs[i,j],ph_symmetry=0,num_bands=10,m_eff=1,sym=True,p_mode=-1,get_wf=True,u_B=0,u_T=0))
        params_tmp = params.copy()
        params_tmp['a'] = [a]
        params_tmp['m0'] = [m0s[i,j]]
        params_tmp['m1'] = [m1]
        params_tmp['D'] = [D]
        params_tmp['v_F'] = [v_F]
        params_tmp['m_z'] = [m_zs[i,j]]
        params_tmp['k'] =[-.015,.015]
        params_tmp['num_bands'] =[8]
        qa_tmp = tools.QuickAdptive(
            funcs.calculate_lead_spectrum,
            learner_bounds_names=['k'],
            arg_picker=itemgetter('mn'),
            **params_tmp
        )

        qa_tmp.learner.strategy = "npoints"

        qa_tmp.runn(
            goal=lambda learner: learner.npoints > 99,
            executor=client
        )
        qa_tmps.append(qa_tmp)
    jobs.append(jobs_tmp)
    qas.append(qa_tmps)

In [ ]:
sel_i = [0,0,0,1,3,3,3]
sel_j = [0,1,2,0,0,1,2]

In [ ]:
nonzeroD_i = [2,6]
DDs = [-4,-14]
mus = [-DDs[i]/m1*(m0s[sel_i[nonzeroD_i[i]],sel_j[nonzeroD_i[i]]]+m_zs[sel_i[nonzeroD_i[i]],sel_j[nonzeroD_i[i]]]) for i in range(2)]
qa_Ds = []
for i in range(2):
    params_tmp = params.copy()
    params_tmp['a'] = [a]
    params_tmp['m0'] = [m0s[sel_i[nonzeroD_i[i]],sel_j[nonzeroD_i[i]]]]
    params_tmp['m1'] = [m1]
    params_tmp['D'] = [DDs[i]]
    params_tmp['v_F'] = [v_F]
    params_tmp['m_z'] = [m_zs[sel_i[nonzeroD_i[i]],sel_j[nonzeroD_i[i]]]]
    params_tmp['k'] =[-.015,.015]
    params_tmp['num_bands'] =[8]
    params_tmp['mu'] = [mus[i]]
    
    qa_D = tools.QuickAdptive(
        funcs.calculate_lead_spectrum,
        learner_bounds_names=['k'],
        arg_picker=itemgetter('mn'),
        **params_tmp
    )
    
    qa_D.learner.strategy = "npoints"
    qa_D.runn(
        goal=lambda learner: learner.npoints > 99,
        executor=client
    )

    qa_Ds.append(qa_D)

In [ ]:
for i in range(len(m0s)):
    for j in range(len(m0s[i])):
        qas[i][j].runner.live_info()

In [ ]:
lead = systems.make_lead(
    a=a, a_z=1, W=W, T=0,
    ph_symmetry=False,
    vector_potential="[0, - B_x * (z - {}), 0]".format(0),
    subst={'Delta': "Delta_lead(delta, x, y, z)",
           'U':   "U_lead(u_B, u_T, x, y, z)"},
    ham_type='2D'        
)

rho = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_0, systems.sigma_0))

In [ ]:
Ess = []
densitys1 = []
densitys2 = []
for i in range(len(m0s)):
    Es_tmp,d1_tmp,d2_tmp = [],[],[]
    for j in range(len(m0s[i])):
        DP = -(m0s[i,j]+m_zs[i,j])*D/m1
        res = jobs[i][j].result()
        Es_tmp.append(res['Es'])
        d1_tmp.append(rho(res['wfs'][:,np.where(res['Es']>0)[0][0]]))
        d2_tmp.append(rho(res['wfs'][:,np.where(res['Es']>0)[0][1]]))
    Ess.append(Es_tmp)
    densitys1.append(d1_tmp)
    densitys2.append(d2_tmp)

In [ ]:
Type = "approx"

fig, ax = plt.subplots(len(m0s),2*len(m0s[0]),figsize=(4*len(m0s[0]),2*len(m0s)),layout='tight')
for i in range(len(m0s)):
    for j in range(len(m0s[i])):
        ax[i,1+2*j].plot(np.zeros(len(Ess[i][j])),Ess[i][j],"kx")
        ax[i,0+2*j].plot(densitys1[i][j],"b")
        ax[i,1+2*j].plot(0,Ess[i][j][np.where(res['Es']>0)[0][0]],"bo")
        ax[i,0+2*j].plot(densitys2[i][j],"r")
        ax[i,1+2*j].plot(0,Ess[i][j][np.where(res['Es']>0)[0][1]],"ro")
        if(Type =="exact"):
            density_a = np.exp(-np.arange(0,W+1,a)*(v_F-np.sqrt(v_F**2-4*(m0s[i,j]+m_zs[i,j])*m1))/(2*m1)) - np.exp(-np.arange(0,W+1,a)*(v_F+np.sqrt(v_F**2-4*(m0s[i,j]+m_zs[i,j])*m1))/(2*m1))
            density_a2= np.exp(-np.arange(0,W+1,a)*(v_F-np.sqrt(v_F**2-4*(m0s[i,j]-m_zs[i,j])*m1))/(2*m1)) - np.exp(-np.arange(0,W+1,a)*(v_F+np.sqrt(v_F**2-4*(m0s[i,j]-m_zs[i,j])*m1))/(2*m1))
        elif(Type =="approx"):
            density_a = np.exp(-np.arange(0,W+1,a)*(m0s[i,j]+m_zs[i,j])/v_F)
            density_a2= np.exp(-np.arange(0,W+1,a)*(m0s[i,j]-m_zs[i,j])/v_F)
        else:
            print("error")
        if(m0s[i][j] + m_zs[i][j]>0):
            ax[i,0+2*j].plot(np.arange(W/a+1),.5*density_a**2/np.sum(density_a**2),"k--",lw=3)
        if(m0s[i][j] - m_zs[i][j]>0):
            ax[i,0+2*j].plot(W/a-np.arange(W/a+1),.5*density_a2**2/np.sum(density_a2**2),"k:",lw=3)
            
        ax[i,0+2*j].set_title("$m_0=$"+str(m0s[i,j])+" eV,"+" $M_z=$"+str(m_zs[i,j])+" eV")
        ax[i,0+2*j].set_ylim(0, 0.02);
        ax[i,1+2*j].set_ylim(-.1, .1);
        if (m0s[i][j]> m_zs[i][j]):
            ax[i,0+2*j].add_patch(Rectangle((-50,0),600,1,color="purple",alpha=.5))
        if (abs(m0s[i][j]) < m_zs[i][j]):
            ax[i,0+2*j].add_patch(Rectangle((-50,0),600,1,color="gold",alpha=.15))
#ax[0].set_xlabel("$W~[a]$");
#ax[1].set_xlabel("$k$");
#ax[0].set_ylabel("$|\Psi|^2$");
#ax[1].set_ylabel("$E~[eV]$");
fig.suptitle("W.f. map "+str(Type))
plt.savefig("wfs_map_"+str(Type)+".png")

In [ ]:
dim = 0.125

posx = np.array([dim,3.2*dim,5.9*dim,dim,dim,3.2*dim,5.9*dim])-dim*.25
#posx = np.array([1/3.5-1/7,2/3.5-1/7,1-1/7,1/3.5-1/7,1/3.5-1/7,2/3.5-1/7,1-1/7])
posy = np.array([7*dim,7*dim,7*dim,5*dim,dim*1.2,dim*1.2,dim*1.2])
size_x=dim*1.3
size_y=dim*1.7

In [ ]:
from matplotlib.patches import Polygon
from matplotlib.patches import FancyArrowPatch

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,1,figsize=(3.5,3),layout='constrained');
ax.set_xticks([]); ax.set_yticks([]);
ax.set_ylim(0.01,.99); ax.set_xlim(0.01,.99);
ax.set_xlabel("$M_z$"); ax.set_ylabel("$m_0$");
colors = ["green","tab:orange","green","tab:orange"]
ls = ["-","--","-","-"]

prop = dict(arrowstyle="-|>,head_width=0.2,head_length=0.5",
                shrinkA=0,shrinkB=0,linewidth=.6)

axs = []
for k in range(len(sel_i)):
    axs.append(ax.inset_axes([posx[k]-size_x/2, posy[k]-size_y/2, size_x, size_y]))  # [x, y, width, height] w.r.t. ax
    #axs[k].plot([9, 0], [0, 9], color="r")
    axs[k].plot(densitys1[sel_i[k]][sel_j[k]],c="green")
    axs[k].plot(densitys2[sel_i[k]][sel_j[k]],c="tab:orange")
    axs[k].set_ylim(0,0.015);
    axs[k].set_xticks([]); axs[k].set_yticks([]);
    ks = qas[sel_i[k]][sel_j[k]].learner.learners[0].to_dataframe()['x'].values
    Es = np.array([qas[sel_i[k]][sel_j[k]].learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))])
    les2 = np.array([Es[i][np.where(Es[i]>0)[0][1]] for i in range(len(ks))])
    les1 = np.array([Es[i][np.where(Es[i]>0)[0][0]] for i in range(len(ks))])
    other_bands = np.array([Es[i][np.append(np.where(Es[i]<0)[0][:],np.where(Es[i]>0)[0][2:])] for i in range(len(ks))])
    less = [les1,les2]
    scaleEs = 1.75
    scaleks = 2.5
#    for i in range(len(ks)):
#        ax.plot([(ks[i]-min(ks))*scaleks+posx[k]+size_x/2+0.02]*len(other_bands[i]),other_bands[i]*scaleEs+posy[k],"k.",markeredgewidth=0,ms=2)
    #ax.plot((ks-min(ks))*scaleks+posx[k]+size_x/2+0.02,*scaleEs+posy[k],"k-",lw=1)
    [ax.plot((ks-min(ks))*scaleks+posx[k]*.98+size_x/2+0.02,other_bands[:,i]*scaleEs+posy[k],"k-",markeredgewidth=0,lw=1) for i in range(len(other_bands[0,:]))]
    [ax.plot((ks-min(ks))*scaleks+posx[k]*.98+size_x/2+0.02,less[i]*scaleEs+posy[k],ls=ls[i],color=colors[i],markeredgewidth=0,lw=1) for i in range(len(less))]

    if k == 1:
        axs[k].annotate("$\psi_\\text{e}^+$",xy=(30,densitys1[sel_i[k]][sel_j[k]][30]),xytext=(100,0.015),fontsize=8,color="k",arrowprops=prop)
        axs[k].annotate("$\psi_\\text{e}^-$",xy=(250,densitys2[sel_i[k]][sel_j[k]][250]),xytext=(275,0.015),fontsize=8,color="k",arrowprops=prop)
    if k == 2:
        axs[k].annotate("$\psi_\\text{e}^+$",xy=(30,densitys1[sel_i[k]][sel_j[k]][30]),xytext=(75,0.02),fontsize=8,color="k",arrowprops=prop)
        axs[k].annotate("$\psi_\\text{s}^{-,+1}$",xy=(250,densitys2[sel_i[k]][sel_j[k]][250]),xytext=(150,0.0125),fontsize=8,color="k",arrowprops=prop)
    if k == 5:
        axs[k].annotate("$\psi_\\text{s}^{-,+1}$",xy=(210,densitys1[sel_i[k]][sel_j[k]][210]),xytext=(10,0.011),fontsize=8,color="k",arrowprops=prop)
        axs[k].annotate("$\psi_\\text{s}^{-,+2}$",xy=(420,densitys2[sel_i[k]][sel_j[k]][420]),xytext=(220,0.0075),fontsize=8,color="k",arrowprops=prop)
        
ax.set_xticks([posx[0],posx[1],posx[2]],["$M_z=0$","$M_z<|m_0|$","$M_z>|m_0|$"],minor=False)
ax.set_xticks([],minor=True)
ax.set_yticks([.5],[0],minor=True)
[axs[i].set_ylim(0,0.025) for i in [0,1,2]]
axs[3].set_xlabel("");

ks = qa_Ds[0].learner.learners[0].to_dataframe()['x'].values
Es = np.array([qa_Ds[0].learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))])+mus[0]
#ax.plot((ks-min(ks))*scaleks+posx[2]*.98+size_x/2+0.02+0.1,Es*scaleEs+posy[2],".",color="r",markeredgewidth=0,ms=2)

colors = ["green","tab:orange","k","k"]
ks = qas[sel_i[2]][sel_j[2]].learner.learners[0].to_dataframe()['x'].values
Es = np.array([qas[sel_i[2]][sel_j[2]].learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))])
les2 = np.array([Es[i][np.where(Es[i]>0)[0][1]] for i in range(len(ks))])
les1 = np.array([Es[i][np.where(Es[i]>0)[0][0]] for i in range(len(ks))])+mus[0]
lesm1 = np.array([Es[i][np.where(Es[i]<0)[0][-1]] for i in range(len(ks))])+mus[0]
other_bands = np.array([Es[i][np.append(np.where(Es[i]<0)[0][:-1],np.where(Es[i]>0)[0][2:])] for i in range(len(ks))])
less = [les1,les2,lesm1]
[ax.plot((ks-min(ks))*scaleks+posx[2]*.98+size_x/2+0.02+0.1,other_bands[:,i]*scaleEs+posy[2],"k-",markeredgewidth=0,lw=1) for i in range(len(other_bands[0,:]))]
[ax.plot((ks-min(ks))*scaleks+posx[2]*.98+size_x/2+0.02+0.1,less[i]*scaleEs+posy[2],ls=ls[i],color=colors[1-i],markeredgewidth=0,lw=1) for i in range(len(less))]

ks = qa_Ds[1].learner.learners[0].to_dataframe()['x'].values
Es = np.array([qa_Ds[1].learner.learners[0].to_dataframe()['extra_data'][i]['Es'] for i in range(len(ks))])
les2 = np.array([Es[i][np.where(Es[i]>0)[0][1]] for i in range(len(ks))])
les1 = np.array([Es[i][np.where(Es[i]>0)[0][0]] for i in range(len(ks))])
other_bands = np.array([Es[i][np.append(np.where(Es[i]<0)[0][:],np.where(Es[i]>0)[0][2:])] for i in range(len(ks))])
less = [les1,les2]
[ax.plot((ks-min(ks))*scaleks+posx[6]*.98+size_x/2+0.02+0.1,other_bands[:,i]*scaleEs+posy[6],"k-",markeredgewidth=0,lw=1) for i in range(len(other_bands[0,:]))]
[ax.plot((ks-min(ks))*scaleks+posx[6]*.98+size_x/2+0.02+0.1,less[i]*scaleEs+posy[6],ls=ls[i],color=colors[i],markeredgewidth=0,lw=1) for i in range(len(less))]


ax.plot([0,.475],[.5,.5],"k--")
ax.plot([.475,.625],[.5,1],"k-")
ax.plot([.475,.625],[.5,0],"k-")
ax.text(0.33,0.62,"QSHI"); ax.text(0.24,0.37,"NI"); ax.text(0.71,0.49,"QAHI")
ax.text(0.50,0.55,"$M_z=m_0$",rotation=68); ax.text(0.495,0.25,"$M_z=-m_0$",rotation=-68); 
ax.text(0.79,0.71,"$D=0$",fontsize=7); ax.text(0.89,0.71,"$D<0$",fontsize=7);
ax.text(0.79,0.32,"$D=0$",fontsize=7);  ax.text(0.89,0.32,"$D<0$",fontsize=7);

ax.text(-.035,0.65,"$m_0>0$",rotation=90); ax.text(-.035,0.15,"$m_0<0$",rotation=90);
ax.add_patch(Polygon([[0,.5],[.475,.5],[.625,1],[0,1]],color="tab:orange",alpha=.2,linewidth=0))
ax.add_patch(Polygon([[.475,.5],[.625,1],[1,1],[1,0],[.625,0]],color="tab:blue",alpha=.2,linewidth=0))

#ax.annotate('', xy=(1.025, .1), xycoords='axes fraction', xytext=(1.025, .9), 
#            arrowprops=dict(arrowstyle="-|>", color='k'))

#ax.text(1.025,0.5,"$\lambda^+$");

plt.savefig("wf_map_final.png",dpi=300)
plt.savefig("wf_map_final.pdf")

## Band spacing

In [ ]:
m0 = 0.01
m1 = 15
D  = 0
m_z = 0.02
W = 500
a=1
v_F=3
params = dict(
    k=[0],
    a=[a],
    a_z=[1],
    W=[W],#
#    W=[500,1000],    
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[0],# - m_z/2*D*m1/abs(D*m1) + 1e-3 ],# ],
    m0=[0.01,0.04,-0.01,-0.04],
    m1=[m1],
    D=[D],
    v_F=[v_F],
    h_mode=[1],
#    m_z=[m_z],
    m_z=[0,0.085],#[0.05],#
    ph_symmetry=[0],
    num_bands=[30],
    m_eff=[1],
    sym=[True],
    p_mode=[-1],
    get_wf=[True],
    u_B=[0],
    u_T=[0]
)

In [ ]:
qa = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m_z'],
#    learner_bounds_names=['W'],
    arg_picker=itemgetter('Es'),
    **params
)

qa.learner.strategy = "npoints"

## 2nd runner

In [ ]:
params2 = params.copy()
params2['m0']=[-.05,.05]
params2['m_z']=[0]
qa2 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m0'],
#    learner_bounds_names=['k'],    
#    loss_per_interval=[uniform_loss],
    arg_picker=itemgetter('Es'),
    **params2
)

qa2.learner.strategy = "npoints"

In [ ]:
qa.load('./data/band_structures')
qa2.load('./data/band_structures')

In [ ]:
qa.runner.task.result()

In [ ]:
import adaptive
adaptive.notebook_extension()

qa.runn(
    goal=lambda learner: learner.npoints > 1200,
    executor=client
)

In [ ]:
qa.runner.live_info()

In [ ]:
import adaptive
adaptive.notebook_extension()

qa2.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)

In [ ]:
qa2.runner.live_info()

In [ ]:
qa.save('./data/band_structures')
qa2.save('./data/band_structures')

In [ ]:
W_plot = qa.learner.learners[0].to_dataframe()['x'].values
Es_plot = qa.learner.learners[0].to_dataframe()['y'].values

DP = -m0*D/m1
gap_plot = [Es_plot[i][np.where(Es_plot[i]>DP)[0][0]]-Es_plot[i][np.where(Es_plot[i]<DP)[0][-1]] for i in range(len(Es_plot))]
gap_plot2 = [Es_plot[i][np.where(Es_plot[i]>DP)[0][2]]-Es_plot[i][np.where(Es_plot[i]<DP)[0][-3]] for i in range(len(Es_plot))]

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(8,5),layout='tight')
for i in [0,1]:
    hyb_gap = np.sqrt(m0**2+(np.pi*v_F/W_plot)**2)

    if(m0*m1>0): 
        ax[i].plot(W_plot,gap_plot,"b-",label="1st gap")
    #    plt.plot(W_plot,gap_plot,"b-",label="1st gap")    
        validity = W_plot>2.5*v_F/m0
        ax[i].plot(W_plot[validity],4*v_F*abs(m0)/np.sqrt(v_F**2-4*m0*m1)*np.exp(-W_plot[validity]*m0/v_F),"k:",lw=3,label="$\dfrac{4v|m_0|}{\sqrt{v^2-4m_0m_1}}\cdot\mathrm{exp(-Wm_0/v)}$")
        ax[i].plot(W_plot,gap_plot2,"r-",label="2nd gap")
        ax[i].plot(W_plot,2*hyb_gap,"r--",label="$2\sqrt{m_0^2+(\pi v/W)^2}$")
    if(m0*m1<0): 
        ax[i].plot(W_plot,gap_plot,"b-",label="1st gap")
        ax[i].plot(W_plot,2*hyb_gap,"b--",label="$2\sqrt{m_0^2+(\pi v/W)^2}$")
    ax[i].set_xlabel("$W~[\AA]$");
ax[0].set_ylabel("Gap [eV]");
ax[1].set_ylabel("log(Gap)");
ax[0].legend();
if(m0>0):
    fig.suptitle("$m_0=$"+str(m0)+" eV, D="+str(D)+" eV$\AA$, $M_z$="+str(m_z)+" $eV$, QSHI")
else:
    fig.suptitle("$m_0=$"+str(m0)+" eV, D="+str(D)+" eV$\AA$, $M_z$="+str(m_z)+" $eV$, NI")

## Plot lowest energy states

In [ ]:
import kwant 

m0_plot=qa2.learner.learners[0].to_dataframe()['x'].values
Es_plot_m0=qa2.learner.learners[0].to_dataframe()['y'].values

DP_m0 = -(m0_plot+0)*D/m1

first_energy_m0 =np.array([Es_plot_m0[i][np.where(Es_plot_m0[i]>DP_m0[i])[0][0]] for i in range(len(m0_plot))])
second_energy_m0=np.array([Es_plot_m0[i][np.where(Es_plot_m0[i]>DP_m0[i])[0][1]] for i in range(len(m0_plot))])
third_energy_m0 =np.array([Es_plot_m0[i][np.where(Es_plot_m0[i]>DP_m0[i])[0][2]] for i in range(len(m0_plot))])


lead = systems.make_lead(
a=a, a_z=1, W=W, T=0,
ph_symmetry=False,
vector_potential="[0, - B_x * (z - {}), 0]".format(0),
subst={'Delta': "Delta_lead(delta, x, y, z)",
       'U':   "U_lead(u_B, u_T, x, y, z)"},
ham_type='2D'        
)

rho = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_z, systems.sigma_0))

particle_type_m0 = []
for i in range(len(Es_plot_m0)):
    particle_type_tmp = []
    for ii in range(len(Es_plot_m0[i])):
        wf_tmp = qa2.runner.learner.learners[0].to_dataframe()['extra_data'][i]['wfs']
        particle_type_tmp.append(np.sum(rho(wf_tmp[:,ii])))
    particle_type_m0.append(particle_type_tmp)

In [ ]:
m_z_plots = []
Es_plots = []
first_energys = []
second_energys = []
third_energys = []
particle_types = []
DPps, DPms = [],[]
if D==0:
    which_m0s = [1,3]
else:
    which_m0s = [1,3]
for j in which_m0s:
    m_z_plots.append(qa.learner.learners[j].to_dataframe()['x'].values)
    Es_plots.append(qa.learner.learners[j].to_dataframe()['y'].values)
    
    
    DPps.append(-(params['m0'][j]+m_z_plots[-1])*D/m1)
    DPms.append(-(params['m0'][j]-m_z_plots[-1])*D/m1)
    first_energys.append( np.array([Es_plots[-1][i][np.where(Es_plots[-1][i]>DPps[-1][i])[0][0]] for i in range(len(m_z_plots[-1]))]))
    second_energys.append(np.array([Es_plots[-1][i][np.where(Es_plots[-1][i]>DPps[-1][i])[0][1]] for i in range(len(m_z_plots[-1]))]))
    third_energys.append( np.array([Es_plots[-1][i][np.where(Es_plots[-1][i]>DPps[-1][i])[0][2]] for i in range(len(m_z_plots[-1]))]))
    
    particle_type = []
    for i in range(len(Es_plots[-1])):
        particle_type_tmp = []
        for ii in range(len(Es_plots[-1][i])):
            wf_tmp = qa.runner.learner.learners[j].to_dataframe()['extra_data'][i]['wfs']
            particle_type_tmp.append(np.sum(rho(wf_tmp[:,ii])))
        particle_type.append(particle_type_tmp)

    particle_types.append(particle_type)

if(D!=0):
    second_energys[0] = np.array([Es_plots[0][i][np.where(Es_plots[0][i]>DPms[0][i])[0][0]] for i in range(len(m_z_plots[0]))])

In [ ]:
#plt.hist(np.array(particle_type_m0).flatten(),30)

In [ ]:
#first_energys[0] = first_energys[0]-DPs[0][0]
#second_energys[0] = second_energys[0]-DPs[0][0]
#third_energys[0] = third_energys[0]-DPs[0][0]

In [ ]:
j=0
for pos in np.arange(len(m_z_plots[j])):
    plt.plot(m_z_plots[j][pos]*np.ones(len(Es_plots[j][pos])),Es_plots[j][pos],"k.",markeredgewidth=0,markersize=5)

if(D!=0):
    C_scale = .015
    cutoff = 1
else:
    C_scale = .05
    cutoff = 2.5

C_scale = .03

plt.ylim(-.1,.1)        
plt.plot(m_z_plots[j], -(params['m0'][which_m0s[j]]+m_z_plots[j])*D/m1,"r-")
plt.plot(m_z_plots[j], -(params['m0'][which_m0s[j]]-m_z_plots[j])*D/m1,"r-")

#plt.plot(m_z_plots[j], (params['m0'][which_m0s[j]]-m_z_plots[j])-0.005,"g-")

m_z_plot = m_z_plots[j]

kappa_m = (params['m0'][which_m0s[j]]-m_z_plot)/v_F*np.sqrt(1-D**2/m1**2)
kappa_p = (params['m0'][which_m0s[j]]+m_z_plot)/v_F*np.sqrt(1-D**2/m1**2)

validityp = (W>.1/kappa_p)
validitym = (W>.1/kappa_m)
plt.plot(m_z_plot[validityp]*f,(C_scale*np.exp(-W*kappa_p)+DPps[j])[validityp]*f,"b--",lw=2,label=E_hyb_p_label)
plt.plot(m_z_plot[validitym]*f,(C_scale*np.exp(-W*kappa_m)+DPms[j])[validitym]*f,"c--",lw=2,label=E_hyb_m_label)
plt.plot(m_z_plot[validitym]*f,(-C_scale*np.exp(-W*kappa_m)+DPms[j])[validitym]*f,"c--",lw=2,label=E_hyb_m_label)

In [ ]:
fig, ax_ = plt.subplots(1,3,figsize=(7,3),layout = 'constrained',sharey=True)
fig.get_layout_engine().set(w_pad=0.015, h_pad=0.0, wspace=0.0, hspace=0.0)
from matplotlib.ticker import FixedLocator, FormatStrFormatter
import matplotlib.ticker as ticker
from matplotlib.patches import Polygon
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list("", ["tab:blue","tab:purple","tab:red"])

ax = []
for i in [0,1,2]:
    if i ==0:
        ax_inset = ax_[i].inset_axes([0.12,0.66,.4,.3])
    elif i == 1 and D==0:
        ax_inset = ax_[i].inset_axes([0.26,0.585,.5,0.375])
    elif i == 1 and D!=0:
        ax_inset = ax_[i].inset_axes([0.15,0.08,.5,0.375])
    else:
        ax_inset = ax_[i].inset_axes([0,0,0.001,0.001])
        ax_inset.set_xticks([]); ax_inset.set_yticks([]);

    ax=ax+[ax_[i],ax_inset]

if(D!=0):
    C_scale = .015
    cutoff = .5
else:
    C_scale = .05
    cutoff = 2.5

min_fade = .2
max_fade = 1
#C_scale = 4*v_F*(m1**2-D**2)*abs(m0)/np.sqrt(m1**3*(v_F**2*m1-4*(m1**2-D**2)*m0))
    
subband_s=np.pi*v_F/W

hyb_gap = np.sqrt(m0_plot**2+subband_s**2)
[ax[i].plot([-1,1],[0,0],"darkgrey",ls="-",lw=.5,zorder=-2) for i in [0,2,4]]
    
if(D!=0):
    ax[0].plot(m0_plot,(DP_m0),"k-",lw=.5)

fades = [1]*3+[.6]+[.2]*6
for n in np.arange(0,10):
    band = np.array([Es_plot_m0[ii][np.where(Es_plot_m0[ii]>DP_m0[ii])[0][n]] for ii in range(len(m0_plot))])
    band_spin = np.array([particle_type_m0[ii][np.where(Es_plot_m0[ii]>DP_m0[ii])[0][n]] for ii in range(len(m0_plot))])

    points = np.array([m0_plot,band]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    norm = plt.Normalize(0, 1)
    
    lc = LineCollection(segments, cmap=cmap, norm=norm, alpha=fades[n], zorder=-1)
    lc.set_array(band_spin)
    lc.set_linewidth(2)
    #ax[0].add_collection(lc)
    ax[0].plot(m0_plot,band,"-",c="tab:purple",lw=2,alpha=fades[n],zorder=-1)    

for n in np.arange(1,6):
    if n > 2:
        fade = min_fade
    else:
        fade = max_fade
    ax[0].plot(m0_plot,np.sqrt(abs(m0_plot)**2+subband_s**2*n**2),"-",c="k",lw=1,alpha=fade, zorder=0)

kappa = m0_plot/v_F*np.sqrt(1-D**2/m1**2)
validity = (W>cutoff/kappa) & (m0_plot>0)
if(D==0):
    E_hyb_label = "$E_{\mathrm{e}}^\pm(M_z=0)$"
else:
    E_hyb_label ="$E_{\mathrm{e}}^\pm+E_\mathrm{DP}(M_z=0)$"
ax[0].plot(m0_plot[validity],(C_scale*np.exp(-W*kappa)+DP_m0)[validity],"--",c="tab:orange",lw=2,label=E_hyb_label)
ax[1].plot([0,0],params2['m0'],"k-",lw=2)
ax[1].plot(params['m_z'],[params['m0'][which_m0s[0]]]*2,"k-",lw=2)
ax[1].plot(params['m_z'],[params['m0'][which_m0s[1]]]*2,"k-",lw=2)
ax[1].add_patch(Polygon([[0,0],[.1,.1],[-.1,.1]],color="tab:orange",alpha=.4,linewidth=0))
ax[1].add_patch(Polygon([[0,0],[.1,.1],[.1,-.1]],color="tab:blue",alpha=.4,linewidth=0))
ax[1].add_patch(Polygon([[0,0],[-.1,.1],[-.1,-.1]],color="tab:blue",alpha=.4,linewidth=0))
ax[1].set_xlim(-.085*.1,.085*1.1); ax[1].set_ylim(np.array(params2['m0'])*1.2)
ax[1].set_xticks([0,0.08],[0,80],minor=False); ax[1].set_yticks(params2['m0'],[int(i*1e3) for i in params2['m0']],minor=False);
ax[1].set_xticks([.04],[""],minor=True); ax[1].set_yticks([.0],[""],minor=True);
ax[1].text(.005,np.mean([params['m0'][which_m0s[0]],params['m0'][which_m0s[1]]])-.005,"a)"); ax[1].text(.035,params['m0'][which_m0s[0]]-0.02,"b)"); ax[1].text(.035,params['m0'][which_m0s[1]]+0.01,"c)")
ax[1].set_xlabel("$M_z$",labelpad=-8); ax[1].set_ylabel("$m_0$",labelpad=-10);

if(D==0):
    E_hyb_p_label = "$E_{\mathrm{e}}^+$"
    E_hyb_m_label = "$E_{\mathrm{e}}^-$"
else:
    E_hyb_p_label = "$E_{\mathrm{e}}^++E_\mathrm{DP}$"
    E_hyb_m_label = "$E_{\mathrm{e}}^-+E_\mathrm{DP}$"
for j in [0,1]:
    m_z_plot = m_z_plots[j]
    Es_plot  = Es_plots[j]
    first_energy  = first_energys[j]
    second_energy = second_energys[j]
    third_energy  = third_energys[j]
    p_type = particle_types[j]
    m0 = params['m0'][which_m0s[j]]
    
    hyb_gap = np.sqrt(m0**2+subband_s**2)
    hyb_gap2 = np.sqrt(m0**2+4*subband_s**2)
    kappa_m = (m0-m_z_plot)/v_F*np.sqrt(1-D**2/m1**2)
    kappa_p = (m0+m_z_plot)/v_F*np.sqrt(1-D**2/m1**2)
    for i in [0,1]:
        if (j==0 and i==1):
            f = 1e3
        else:
            f = 1        
        #if(j==0 and i==1):
        #    ax[2+j*2+i].plot(m_z_plot*f,(first_energy-DPps[j])*f,".",c="k",ms=1)
        #    ax[2+j*2+i].plot(m_z_plot*f,(second_energy-DPms[j])*f,".",c="k",ms=1)
        #else:          
        for n in np.arange(0,10):
            if((j==0 and i==1 and n<2) or (j!=0 or i!=1)):
                band = np.array([Es_plot[ii][np.where(Es_plot[ii]>DPps[j][ii])[0][n]] for ii in range(len(m_z_plot))])
                band_spin = np.array([p_type[ii][np.where(Es_plot[ii]>DPps[j][ii])[0][n]] for ii in range(len(m_z_plot))])
    
                points = np.array([m_z_plot*f,band*f]).T.reshape(-1, 1, 2)
                segments = np.concatenate([points[:-1], points[1:]], axis=1)
                norm = plt.Normalize(-1, 1)
                
                lc = LineCollection(segments, cmap=cmap, norm=norm, alpha=fades[n],zorder=-1)
                lc.set_array(band_spin)
                lc.set_linewidth(2)
                lc.set_linestyle('solid')
                ax[2+j*2+i].add_collection(lc)

                if(n == 1):
                    lc_to_plot = lc
                    
            if(D!=0):
                ax[2+j*2+i].plot(m_z_plot*f,DPps[j]*f,"k-",lw=.5,label="$E_\mathrm{DP}$")
                ax[2+j*2+i].plot(m_z_plot*f,DPms[j]*f,"k-",lw=.5,label="$E_\mathrm{DP}$")

        if(m0*m1>0):
            validityp = (W>cutoff/kappa_p)
            validitym = (W>cutoff/kappa_m) & (m_z_plot<m0)
            if(i==0):
                #continue
                ax[2+j*2+i].plot(m_z_plot[validityp]*f,(C_scale*np.exp(-W*kappa_p)+DPps[j])[validityp]*f,"--",c="green",lw=2,label=E_hyb_p_label)
                ax[2+j*2+i].plot(m_z_plot[validitym]*f,(C_scale*np.exp(-W*kappa_m)+DPms[j])[validitym]*f,"--",c="tab:orange",lw=2,label=E_hyb_m_label)
            else:
                #continue
                ax[2+j*2+i].plot(m_z_plot[validityp]*f,(C_scale*np.exp(-W*kappa_p))[validityp]*f,"--",c="green",lw=2,label=E_hyb_p_label)
                ax[2+j*2+i].plot(m_z_plot[validitym]*f,(C_scale*np.exp(-W*kappa_m))[validitym]*f,"--",c="tab:orange",lw=2,label=E_hyb_m_label)
        else:        
            validity = (W>cutoff/kappa_p) & (m_z_plot>hyb_gap)
            ax[2+j*2+i].plot(m_z_plot[validity],(C_scale*np.exp(-W*kappa_p)+DPps[j])[validity],"--",c="green",lw=2,label=E_hyb_p_label)
            ax[2+j*2+i].plot([0],[-1],"--",c="tab:orange",lw=2,label=E_hyb_m_label)
        fade = max_fade
        if i == 0:
            for n in np.arange(1,6):
                band_p = np.sqrt((abs(m0)+m_z_plot)**2+subband_s**2*n**2)
                band_m = np.sqrt((abs(m0)-m_z_plot)**2+subband_s**2*n**2)
                ax[2+j*2+i].plot(m_z_plot,band_m,"-",c="k",lw=1,alpha=fade,zorder=0)
                if n > 1:
                    fade = min_fade
                else:
                    fade = max_fade

                ax[2+j*2+i].plot(m_z_plot,band_p,"-",c="k",lw=1,alpha=fade,zorder=0)
                
                prop = dict(arrowstyle="-|>,head_width=0.2,head_length=0.5",
                shrinkA=0,shrinkB=0,linewidth=.6)

                if j==1 and i == 0:
                    if n == 1:
                        ax[2+j*2+i].annotate("$\\psi_\\text{s}^{+,+1}$",xy=(m_z_plot[10],band_p[10]),xytext=(m_z_plot[10],band_p[10]-0.025), arrowprops=prop);
                        ax[2+j*2+i].annotate("$\\psi_\\text{s}^{-,+1}$",xy=(m_z_plot[110],band_m[110]),xytext=(m_z_plot[110]-0.015,band_m[110]-0.0125), arrowprops=prop);
                    if n == 2:
                        ax[2+j*2+i].annotate("$\\psi_\\text{s}^{-,+2}$",xy=(m_z_plot[40],band_m[40]),xytext=(m_z_plot[40]+0.0125,band_m[40]-0.0035), arrowprops=prop);

ax[0].set_ylabel("$E$ [meV]");
ax[0].set_xlabel("$m_0$ [meV]"); ax[2].set_xlabel("$M_z$ [meV]"); ax[4].set_xlabel("$M_z$ [meV]");
ax[0].set_xlim(params2['m0']); ax[2].set_xlim(params['m_z']); ax[4].set_xlim(params['m_z']);
ax[0].set_xticks(np.arange(-.04,.041,.02),np.arange(-40,41,20),minor=False);

ax[0].set_yticks(np.arange(0,0.16,0.02),np.arange(0,160,20),minor=False);
ax[2].set_xticks(np.arange(0,0.1,0.02),np.arange(0,100,20),minor=False);
ax[4].set_xticks(np.arange(0,0.1,0.02),np.arange(0,100,20),minor=False);

ax[4].plot([0,0],[-1,-1],"k-",lw=1,label="$E_{\\text{s}}^{\\pm,+n}$");

if(D==0):
    ax[0].set_ylim(-0.005,.09)
else:
    ax[0].set_ylim(-.09,.09)
    
ax[3].set_xlabel("$M_z$ [meV]",labelpad=-8);
if(D==0):
     ax[3].set_ylabel("$E$ [meV]",labelpad=1);
else:
    ax[3].set_ylabel("$E-E_\mathrm{DP}$ [meV]",labelpad=1);
ax[3].set_xticks([0,80],minor=False); ax[3].set_xticks([20,40,60],minor=True);
#ax[3].yaxis.set_major_locator(ticker.LogLocator(base=10, numticks=2))

myticks = [1e2,1,1e-2,1e-4,1e-6]

ax[3].set_yscale('log')
ax[3].set_ylim(1e-6,1e2)

ax[3].yaxis.set_major_locator(FixedLocator(myticks))

ax[3].yaxis.set_minor_locator(ticker.LogLocator(base=100,
                                             subs=(.1,.3,.5,.7,.9),
                                             numticks=16))
ax[3].yaxis.set_minor_formatter(ticker.NullFormatter())

ax[0].legend(loc='center left',frameon=True);
ax[4].legend(loc='upper right',frameon=True);

cbar = fig.colorbar(lc_to_plot, ax = ax[4])
cbar.set_label("$\\langle\\sigma_z\\rangle$", labelpad=-1)
ax[0].add_patch(Rectangle((0,-.1),.06,2,color="tab:orange",alpha=.2,linewidth=0,zorder=-2))
ax[2].add_patch(Rectangle((0,-.1),params['m0'][which_m0s[0]],2,color="tab:orange",alpha=.2,linewidth=0,zorder=-2))
ax[2].add_patch(Rectangle((params['m0'][which_m0s[0]],-.1),.1,2,color="tab:blue",alpha=.2,linewidth=0,zorder=-2))
ax[4].add_patch(Rectangle((abs(params['m0'][which_m0s[1]]),-.1),.1,2,color="tab:blue",alpha=.2,linewidth=0,zorder=-2))

ax[0].add_patch(Rectangle((-0.06,0.05),.065,.05,facecolor="white",edgecolor='k',alpha=.8,linewidth=.5,linestyle="-",zorder=2))
ax[2].add_patch(Rectangle((-0.01,0.0425),.0775,.05,facecolor="white",edgecolor='k',alpha=.8,linewidth=.5,linestyle="-",zorder=2))

ax[0].text(params2['m0'][0]+(params2['m0'][1]-params2['m0'][0])*.075,0.0075,"NI"); 
ax[0].text(params2['m0'][0]+(params2['m0'][1]-params2['m0'][0])*.775,0.0075,"QSHI"); 

ax[2].text(params['m_z'][0]+(params['m_z'][1]-params['m_z'][0])*.075,0.0075,"QSHI"); 
ax[2].text(params['m_z'][0]+(params['m_z'][1]-params['m_z'][0])*.775,0.0075,"QAHI"); 

ax[4].text(params['m_z'][0]+(params['m_z'][1]-params['m_z'][0])*.075,0.0075,"NI"); 
ax[4].text(params['m_z'][0]+(params['m_z'][1]-params['m_z'][0])*.775,0.0075,"QAHI"); 

caption = ['(a)','(b)','(c)']
for i in range(len(caption)):
    if i == 0:
        sshift =-0.175
    else:
        sshift =-0.1
    ax[i*2].text(
        sshift, .985, 
        caption[i],
        transform=ax[i*2].transAxes
    )
plt.savefig("lowest_energy_D"+str(D)+".png",bbox_inches='tight',dpi=300)
plt.savefig("lowest_energy_D"+str(D)+".pdf",bbox_inches='tight')

In [ ]:
m0ss = np.array([120,41,10,17,3,77,14,1.5,3,0,127,32,3.5,9,6])

In [ ]:
np.mean(m0ss), np.mean(m0ss[m0ss<60])

## Plot lowest energy states zoom - QSHI

In [ ]:
m0 = 0.01
m1 =  15
D  = 0
m_z = 0
W = 1000
a=1
v_F=3
params = dict(
    k=[0],
    a=[a],
    a_z=[a],
    W=[W],
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[-100],
    h_mode=[1],
    m_z=[0,0.015],
    ph_symmetry=[0],
    num_bands=[50],
    m_eff=[1],
    sym=[True],
    p_mode=[-1],
    get_wf=[False],
    u_B=[0],
    u_T=[0]
)

In [ ]:
params3 = dict(params)
params3['p_mode']=[2]
params3['T']=[4]
params3['sym']=[True]
qa3 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('Es'),
    **params3
)
params4 = dict(params)
params4['p_mode']=[2]
params4['T']=[4]
params4['sym']=[False]
qa4 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('Es'),
    **params4
)
params5 = dict(params)
params5['p_mode']=[2]
params5['T']=[5]
params5['sym']=[False]
qa5 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('Es'),
    **params5
)
qa3.learner.strategy = "npoints"
qa4.learner.strategy = "npoints"
qa5.learner.strategy = "npoints"

In [ ]:
params33 = dict(params3)
params33['k']=[-.015,.015]
params33['m_z']=[0.002]
qa33 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params33
)
params44 = dict(params4)
params44['k']=[-.015,.015]
params44['m_z']=[0.002]
qa44 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params44
)
params55 = dict(params5)
params55['k']=[-.015,.015]
params55['m_z']=[0.006]
qa55 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params55
)
qa33.learner.strategy = "npoints"
qa44.learner.strategy = "npoints"
qa55.learner.strategy = "npoints"

In [ ]:
import adaptive
adaptive.notebook_extension()

qa3.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)
qa4.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)
qa5.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)
qa33.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)
qa44.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)
qa55.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)

In [ ]:
qa3.runner.live_info()
qa4.runner.live_info()
qa5.runner.live_info()
qa33.runner.live_info()
qa44.runner.live_info()
qa55.runner.live_info()

In [ ]:
x_plots = []
Es_plots = []
DPps, DPms = [],[]
qas = [qa3,qa4,qa5,qa33,qa44,qa55]
H_params = [funcs.H_eff_parameters[15],funcs.H_eff_parameters[15],funcs.H_eff_parameters[16]]
mus = [-p['D']/p['B']*p['Delta']/2 for p in H_params]
mus[0] = 0
for qa_ in qas:
    x_plots.append(qa_.learner.to_dataframe()['x'].values)
    Es_plots.append(qa_.learner.to_dataframe()['y'].values)

x_plots=np.array(x_plots)
Es_plots=np.array(Es_plots)

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(3.5,3),layout = 'constrained',sharey=True)
fig.get_layout_engine().set(w_pad=0.01, h_pad=0.02, wspace=0.0, hspace=0.0)
from matplotlib.ticker import FixedLocator, FormatStrFormatter
import matplotlib.ticker as ticker
from matplotlib.patches import Polygon

if(D==0):
    E_hyb_p_label = "$E_{\mathrm{e}}^+$"
    E_hyb_m_label = "$E_{\mathrm{e}}^-$"
else:
    E_hyb_p_label = "$E_{\mathrm{e}}^++E_\mathrm{DP}$"
    E_hyb_m_label = "$E_{\mathrm{e}}^-+E_\mathrm{DP}$"
for j in range(3):
    m_z_plot = x_plots[j]
    Es_plot  = Es_plots[j]
    k_plot   = x_plots[3+j]
    Esk_plot = Es_plots[3+j]
    if(D!=0):
        C_scale = .015
        cutoff = .5
    else:
        C_scale = .05
        cutoff = 2.5
        
    f = 1
    for pos in np.arange(len(m_z_plot)):
        mzs = m_z_plot[pos]*np.ones(len(Es_plot[pos]))
        if params['mu'][0] == -100 and j>0:
            Ess = Es_plot[pos]+H_params[j]['D']/H_params[j]['B']*(mzs)
        else:
            Ess = Es_plot[pos]
        ax[0,j].plot(mzs*f,Ess*f,"k.",ms=1.5,markeredgewidth=0)
#        ax[0,j].plot(m_z_plot[pos]*f,Ess[25],"r.",ms=4,markeredgewidth=0)
    for pos in np.arange(len(k_plot)):
        ks = k_plot[pos]*np.ones(len(Esk_plot[pos]))
        mzs = .002
        if j == 2:
            mzs = .006
        if params['mu'][0] == -100 and j>0:
            Ess = Esk_plot[pos]+H_params[j]['D']/H_params[j]['B']*(mzs)
        else:
            Ess = Esk_plot[pos]
        ax[1,j].plot(ks*f,Ess*f,"k.",ms=1.5,markeredgewidth=0)
    ax[0,j].plot([mzs]*2,[-1,1],"k:",lw=.5)    
    if j>0:
        ax[0,j].plot(m_z_plot*f,(H_params[j]['D']/H_params[j]['B']*(m_z_plot))*f,"--",color="tab:green",lw=.5,label="$E_\mathrm{DP}^+$")
        ax[0,j].plot(m_z_plot*f,(H_params[j]['D']/H_params[j]['B']*(-m_z_plot))*f,":",color="tab:green",lw=.5,label="$E_\mathrm{DP}^-$")
        ax[1,j].plot([-1,1],[(H_params[j]['D']/H_params[j]['B']*(mzs))*f]*2,"--",color="tab:green",lw=.5,label="$E_\mathrm{DP}^+$")
        ax[1,j].plot([-1,1],[(H_params[j]['D']/H_params[j]['B']*(-mzs))*f]*2,":",color="tab:green",lw=.5,label="$E_\mathrm{DP}^-$")
    else:
        ax[0,j].plot(m_z_plot*f,np.zeros(len(m_z_plot)),"--",color="tab:green",lw=.5,label="$E_\mathrm{DP}^+$")
        ax[0,j].plot(m_z_plot*f,np.zeros(len(m_z_plot)),":",color="tab:green",lw=.5,label="$E_\mathrm{DP}^-$")
        ax[1,j].plot([-1,1],[0]*2,"--",color="tab:green",lw=.5,label="$E_\mathrm{DP}^+$")
        ax[1,j].plot([-1,1],[0]*2,":",color="tab:green",lw=.5,label="$E_\mathrm{DP}^-$")
    ax[0,j].set_xticks(np.arange(0,.1,0.010),np.arange(0,100,10));
    ax[0,j].set_yticks(np.arange(-.1,.1,0.01),np.arange(-100,100,10),minor=False);
    ax[0,j].set_xlim(params3['m_z']);
    ax[0,j].set_ylim(-0.02,0.02)
    #ax[0,j].set_ylim(-0.02,0.06)
    ax[0,j].add_patch(Rectangle((0,-1),abs(H_params[j]['Delta']/2),2,color="tab:orange",alpha=.15,linewidth=0))
    ax[0,j].add_patch(Rectangle((abs(H_params[j]['Delta']/2),-1),.1,2,color="tab:blue",alpha=.15,linewidth=0))
    ax[0,j].set_xlabel("$M_z$ [meV]",labelpad=0);

    ax[1,j].set_xlim(params33['k']);
    ax[1,j].set_xlabel("$k$ [$1/\\AA$]",labelpad=0);
    ax[1,j].set_xticks([-.01,0,.01]);
    les1pos = [23,23,22][j]
    zero_pos = np.where(abs(k_plot)<1e-6)[0][0]
    for shift in [0,2]:
        if params['mu'][0] == -100 and j>0:
            ax[1,j].add_patch(Rectangle((-1,Esk_plot[zero_pos][les1pos+shift]+H_params[j]['D']/H_params[j]['B']*(mzs)),2,Esk_plot[zero_pos][les1pos+shift+1]-Esk_plot[zero_pos][les1pos+shift],color="tomato"))
        else:
            ax[1,j].add_patch(Rectangle((-1,Esk_plot[zero_pos][les1pos+shift]),2,Esk_plot[zero_pos][les1pos+shift+1]-Esk_plot[zero_pos][les1pos+shift],color="tomato"))
    if params['mu'][0] == -100 and j>0:
        ax[1,j].text(.01,Esk_plot[zero_pos][les1pos+1]+H_params[j]['D']/H_params[j]['B']*(mzs)+.0015,"$E_\\text{b}$",fontsize=7)
        ax[1,j].text(.01,Esk_plot[zero_pos][les1pos+2+1]+H_params[j]['D']/H_params[j]['B']*(mzs)+.0015,"$E_\\text{a}$",fontsize=7)
    else:
        ax[1,j].text(.01,Esk_plot[zero_pos][les1pos+1]+.0015,"$E_\\text{b}$",fontsize=7)
        ax[1,j].text(.01,Esk_plot[zero_pos][les1pos+2+1]+.0015,"$E_\\text{a}$",fontsize=7)
#ax[4].set_title("$D\\neq0$\n$\\tilde{E}_\\text{e}^- < \\delta E_\\text{DP}$")
ax[0,0].set_ylabel("$E$ [meV]",labelpad=0);
ax[1,0].set_ylabel("$E$ [meV]",labelpad=0);
#ax[0].set_yticks(np.arange(-.1,.1,0.01),np.arange(-100,100,10),minor=True);

#ax[4].set_xticks(np.arange(0,0.1,0.02),np.arange(0,100,20),minor=True);

#ax[2].set_yticklabels([]); ax[4].set_yticklabels([]);
multiplier = 1.45
qah_space = 0.0075

#for j in range(3):
    #ax[j*2].set_xlim(0,0.04);
    #ax[j*2].set_ylim(-Ds[j]*m0s[j]/m1-(m0s[j]+qah_space)*3,-Ds[j]*m0s[j]/m1+(m0s[j]+qah_space)*3)
    #ax[j*2].set_ylim(0.015,-Ds[j]*m0s[j]/m1+(m0s[j]+qah_space)*3)
    #ax[j*2].set_title("$D\\neq0$\n$m_0=$"+str(int(m0s[j]*1e3))+" meV");
#for j in [1,2]: 
    #ax[j*2].plot([(np.pi*H_params[i]['v_F']/W)**2/(-4*H_params[j]['Delta']/2)]*2,[-1e3,1e3],"b--");
    #ax[j*2].plot([(np.pi*H_params[i]['v_F']/W)**2/(-H_params[j]['Delta']/2)]*2,[-1e3,1e3],"r--");
    #ax[j*2].plot(np.linspace(0,0.08,100),np.sqrt((m0s[j]-np.linspace(0,0.08,100))**2+0.01885**2),"b-");
    #ax[j*2].plot(np.linspace(0,0.08,100),np.sqrt((m0s[j]-np.linspace(0,0.08,100))**2+4*0.01885**2),"b-");

ax[0,0].set_title(str(params3['T'][0])+" QL Sb$_2$Te$_3$\n$D=0$");
ax[0,1].set_title(str(params4['T'][0])+" QL Sb$_2$Te$_3$\n$D<0,\,|D|\\approx|m_1|$");
ax[0,2].set_title(str(params5['T'][0])+" QL Sb$_2$Te$_3$\n$D<0,\,|D|\\approx|m_1|$"); 


ax[0,0].text(.00225,-.011,'(d)');
ax[0,1].text(.00225,-.011,'(e)');
ax[0,2].text(.00625,-.015,'(f)');
caption = [['(a)','(b)','(c)'],['(d)','(e)','(f)']]
ax[0,0].legend(loc='upper right',frameon=True,fontsize=7);
for i in range(len(caption)):
    for j in range(len(caption[0])):
        ax[i,j].text(
            [-.4,-.18,-.18][j], [1.25,.9][i],
            caption[i][j],
            transform=ax[i,j].transAxes
        )
plt.savefig("lowest_energy_QSHI.png",dpi=300,bbox_inches='tight')
plt.savefig("lowest_energy_QSHI.pdf",bbox_inches='tight')

## Create FOM spectra plot

In [ ]:
m0  = -0.015
m1  = 15
D   = 0
m_z = 0
W   = 500
a=1
v_F=3
params = dict(
    k=[-.03,.03],
    a=[a],
    a_z=[1],
    W=[W],#
#    W=[500,1000],    
    T=[0],    
    delta=[0],
    flux=[0],
    mu=[0],# - m_z/2*D*m1/abs(D*m1) + 1e-3 ],# ],
    m0=[m0],
    m1=[m1],
    D=[D],
    v_F=[v_F],
    h_mode=[1],
#    m_z=[m_z],
    m_z=[m_z],#[0.05],#
    ph_symmetry=[0],
    num_bands=[10],
    m_eff=[1],
    sym=[True],
    p_mode=[-1],
    get_wf=[False],
    u_B=[0],
    u_T=[0]
)

In [ ]:
qaFOM1 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params
)
params11 = params.copy()
params11['m_z']=[0.005]

qaFOM11 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params11
)

params2 = params.copy()
params2['m_z']=[0.03]
qaFOM2 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params2
)

params3 = params2.copy()
params3['mu']=[0.017]
qaFOM3 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params3
)

params4 = params3.copy()
params4['delta']=[0.02]
params4['ph_symmetry']=[True]
params4['num_bands']=[8]
params4['get_wf']=[True]

qaFOM4 = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params4
)

qaFOM1.learner.strategy = "npoints"
qaFOM11.learner.strategy = "npoints"
qaFOM2.learner.strategy = "npoints"
qaFOM3.learner.strategy = "npoints"
qaFOM4.learner.strategy = "npoints"

In [ ]:
qaFOM1.load('./data/band_structures')
qaFOM11.load('./data/band_structures')
qaFOM2.load('./data/band_structures')
qaFOM3.load('./data/band_structures')
qaFOM4.load('./data/band_structures')

In [ ]:
#qaFOM1.runner.task.result()

In [ ]:
npoints = 500

In [ ]:
import adaptive
adaptive.notebook_extension()

qaFOM1.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client
)
qaFOM11.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client
)
qaFOM2.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client
)
qaFOM3.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client
)
qaFOM4.runn(
    goal=lambda learner: learner.npoints > npoints,
    executor=client
)

In [ ]:
qaFOM1.runner.live_info()
qaFOM11.runner.live_info()
qaFOM2.runner.live_info()
qaFOM3.runner.live_info()
qaFOM4.runner.live_info()

In [ ]:
qaFOM1.save('./data/band_structures')
qaFOM11.save('./data/band_structures')
qaFOM2.save('./data/band_structures')
qaFOM3.save('./data/band_structures')
qaFOM4.save('./data/band_structures')

In [ ]:
cmap = LinearSegmentedColormap.from_list("", ["tab:blue","black","tab:red"])

In [ ]:
k1 = qaFOM1.runner.learner.to_dataframe()['x'].values
Es1 = qaFOM1.runner.learner.to_dataframe()['y'].values
k11 = qaFOM11.runner.learner.to_dataframe()['x'].values
Es11 = qaFOM11.runner.learner.to_dataframe()['y'].values
k2 = qaFOM2.runner.learner.to_dataframe()['x'].values
Es2 = qaFOM2.runner.learner.to_dataframe()['y'].values
k3 = qaFOM3.runner.learner.to_dataframe()['x'].values
Es3 = qaFOM3.runner.learner.to_dataframe()['y'].values
k4 = qaFOM4.runner.learner.to_dataframe()['x'].values
Es4 = qaFOM4.runner.learner.to_dataframe()['y'].values

lead = systems.make_lead(
    a=a, a_z=1, W=W, T=0,
    ph_symmetry=True,
    vector_potential="[0, - B_x * (z - {}), 0]".format(0),
    subst={'Delta': "Delta_lead(delta, x, y, z)",
           'U':   "U_lead(u_B, u_T, x, y, z)"},
    ham_type='2D'        
)
import kwant 

rho = kwant.operator.Density(lead.finalized(), np.kron(np.kron(systems.sigma_z, systems.sigma_0), systems.sigma_0))

particle_type = []
for i in range(len(Es4)):
    particle_type_tmp = []
    for j in range(len(Es4[i])):
        wf_tmp = qaFOM4.runner.learner.to_dataframe()['extra_data'][i]['wfs']
        particle_type_tmp.append(np.sum(rho(wf_tmp[:,j])))
    particle_type.append(particle_type_tmp)

particle_type = np.array(particle_type)    

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,1.5),layout = 'constrained')

ax.plot([-.1,.5],[0,0],"darkgrey",ls="-",lw=.5);

for i in np.arange(len(k11)):
    ax.plot(k11[i]*np.ones(len(Es11[i]))+.015,Es11[i]+(params11['m0'][0]+params11['m_z'][0])*params11['D'][0]/params11['m1'][0],'.',c='grey',ms=2,markeredgewidth=0)

for i in np.arange(len(k1)):
    ax.plot(k1[i]*np.ones(len(Es1[i])),Es1[i]+(params['m0'][0]+params['m_z'][0])*params['D'][0]/params['m1'][0],'k.',ms=3,markeredgewidth=0)

middle_1 = np.where(k1 == np.min(abs(k1)))[0][0]
middle_11 = np.where(k11 == np.min(abs(k11)))[0][0]
middle_2 = np.where(k2 == np.min(abs(k2)))[0][0]

left_11 = int(middle_11*1.1)
left_2 = int(middle_2*1.1)

prop_trans = dict(arrowstyle="-|>,head_width=0.2,head_length=0.6",
    shrinkA=0,shrinkB=0,linewidth=.8,color="grey")

ax.annotate("",xy=(k11[middle_11]+.015,np.min(Es11[middle_11][np.where(Es11[middle_11]>0)[0][0]])),xytext=(k1[middle_1],np.min(Es1[middle_1][np.where(Es1[middle_1]>0)[0][0]])), arrowprops=prop_trans,zorder=-1)
ax.annotate("",xy=(k11[middle_11]+.015,np.min(Es11[middle_11][np.where(Es11[middle_11]>0)[0][1]])),xytext=(k1[middle_1],np.min(Es1[middle_1][np.where(Es1[middle_1]>0)[0][0]])), arrowprops=prop_trans,zorder=-1)

ax.annotate("",xy=(k11[middle_11]+.015,np.min(Es11[middle_11][np.where(Es11[middle_11]<0)[0][-1]])),xytext=(k1[middle_1],np.min(Es1[middle_1][np.where(Es1[middle_1]<0)[0][-1]])), arrowprops=prop_trans,zorder=-1)
ax.annotate("",xy=(k11[middle_11]+.015,np.min(Es11[middle_11][np.where(Es11[middle_11]<0)[0][-2]])),xytext=(k1[middle_1],np.min(Es1[middle_1][np.where(Es1[middle_1]<0)[0][-2]])), arrowprops=prop_trans,zorder=-1)

ax.annotate("$\\psi_\\text{s}^{-,+1}$",xy=(k11[left_11]+.015,np.min(Es11[left_11][np.where(Es11[left_11]>0)[0][0]])),xytext=(k11[left_11]+.015+0.005,np.min(Es11[left_11][np.where(Es11[left_11]>0)[0][0]])-0.01), arrowprops=prop)
ax.annotate("$\\psi_\\text{s}^{+,+1}$",xy=(k11[left_11]+.015,np.min(Es11[left_11][np.where(Es11[left_11]>0)[0][1]])),xytext=(k11[left_11]+.015+0.005,np.min(Es11[left_11][np.where(Es11[left_11]>0)[0][1]])-0.002), arrowprops=prop)

ax.annotate("$\\psi_\\text{s}^{-,+2}$",xy=(k2[left_2]+.05,np.min(Es2[left_2][np.where(Es2[left_2]>0)[0][1]])),xytext=(k2[left_2]+.05+0.01,np.min(Es2[left_2][np.where(Es2[left_2]>0)[0][1]])-0.01), arrowprops=prop)

ax.text(k2[middle_11]+.056,np.min(Es2[middle_2][np.where(Es2[middle_2]>0)[0][0]])+0.005,"$\\psi_\\text{e}^+$")

for i in np.arange(len(k2)):
    ax.plot(k2[i]*np.ones(len(Es2[i]))+.05,Es2[i]+(params2['m0'][0]+params2['m_z'][0])*params2['D'][0]/params2['m1'][0],'k.',ms=2,markeredgewidth=0)

for i in np.arange(len(k3)):
    ax.plot(k3[i]*np.ones(len(Es3[i]))+.1,Es3[i]+(params3['m0'][0]+params3['m_z'][0])*params3['D'][0]/params3['m1'][0],'k.',ms=2,markeredgewidth=0)

gap_region = .01
correction = 0.002

for i in np.arange(len(k4)):
    for j in np.arange(len(Es4[i])):
        ax.plot(k4[i]+.15,Es4[i][j],'.',c=cmap((particle_type[i,j]+1)/2),ms=2,markeredgewidth=0)

ax.text(-0.02,-0.035,"NI");
ax.text(.073,-0.035,"QAHI");
ax.text(0.165,-0.018,"TSC");

ax.set_xlabel("$k$");
ax.set_ylabel("$E$ [meV]");

ax.set_xticks([0,.05,.1,.15],major=True);
ax.set_yticks(np.arange(-.1,.1,.02),np.arange(-100,100,20));
ax.set_xticklabels([]);

ax.set_xlim(-.025,.175)
ax.set_ylim(-.04,.04)
ax.grid(axis='x');

plt.savefig("FOM_components.png")
plt.savefig("FOM_components.pdf",bbox_inches='tight')

# Chiral Majorana edge states

In [ ]:
syst = systems.make_lead(a=10,a_z=10,W=20,T=10,L=20,ham_type='2D')

In [ ]:
import kwant
kwant.plotter.set_engine("plotly")
#kwant.plotter.set_engine("matplotlib")
import matplotlib.pyplot as plt, matplotlib.backends
plot = kwant.plot(syst, show=False)
plot.show()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
M_zs = qa.learner.to_dataframe()["x"]

In [ ]:
datapoints = qa.learner.to_dataframe()["y"].to_numpy()

In [ ]:
for i in np.arange(0,len(M_zs)):
    plt.plot(np.ones(params['num_bands'])*M_zs[i],datapoints[i],"k.",markersize=.5)

#plt.xlim(0,0.15);
#plt.ylim(-0.04,0.04);
plt.plot([0,0.15],[0.005,0.005],"r--")
plt.plot([0,0.15],[-0.005,-0.005],"r--")
plt.xlabel("$M_z$");
plt.ylabel("$E$");
plt.savefig("bands_trace_W1000_mu0.png")
#plt.savefig("bands_trace_W1000_mu=0p5m0.png")

In [ ]:
A_0 = 3
B_0 = 3
C_0 = 0
C_1 = 0
C_2 = 0
M_0 = -0.3
M_1 = 15
M_2 = 15

In [ ]:
params_bands = dict(
    k=[-.5,.5],
    a=[10],
    a_z=[-20],
    W=[500],
    delta=[0],
    flux=[0],
    mu=[-M_0*C_1/M_1],
    m0=[0],
    m1=[0],
    D=[0],
    h_mode=[3],
    T=[np.round(i) for i in np.array([25])/1.08],#np.array([20,100])/1.187,
    m_z=[0, 0.03, 0.1],
    ph_symmetry=[0],
    num_bands=[50],
    m_eff=[1],
    p_mode=[10],
    get_wf=[False],
    sym = [True],
    A_0 = [A_0],
    B_0 = [B_0],
    C_0 = [C_0],
    C_1 = [C_1],
    C_2 = [C_2],
    M_0 = [M_0],
    M_1 = [M_1],
    M_2 = [M_2]
#    A_perp = [3],
#    A_z = [1]
)

In [ ]:
qa_bands = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('mn'),
    **params_bands
)

qa_bands.learner.strategy = "npoints"

In [ ]:
qa_bands.load('data/band_structures/')

In [ ]:
# import adaptive
adaptive.notebook_extension()

qa_bands.runn(
        goal=lambda learner: learner.npoints > len(params['m_z'])*1000,
        executor=client
    )

In [ ]:
qa_bands.save('data/band_structures/')

In [ ]:
out = qa_bands.runner.learner.plot(plotter=get_plotter(xlabel='k',xlim=(-.5,.5),ylim=(-.2,.2)))

In [ ]:
out

In [ ]:
params = dict(
    k=[0],
    a=[10],
    a_z=[-20],
    W=[500,2000],
    delta=[0],
    flux=[0],
    mu=[-M_0*C_1/M_1],
    m0=[0],
    m1=[0],
    D=[0],
    h_mode=[3],
    T=[np.floor(20/1.08), np.ceil(100/1.08)],#np.array([20,100])/1.187,
    m_z=[0],
    ph_symmetry=[0],
    num_bands=[200],
    m_eff=[1],
    p_mode=[10],
    get_wf=[False],
    sym = [True],
    A_0 = [A_0],
    B_0 = [B_0],
    C_0 = [C_0],
    C_1 = [C_1],
    C_2 = [C_2],
    M_0 = [M_0],
    M_1 = [M_1],
    M_2 = [M_2]
)

In [ ]:
params = dict(
    k=[1e-8],
    a=[10],
    a_z=[-20],
    W=[500],
    delta=[0],
    flux=[0],
    mu=[-M_0*C_1/M_1],
    m0=[0],
    m1=[0],
    D=[0],
    h_mode=[3],
    T=[np.round(i) for i in np.array([25,40,70])/1.08],#np.array([20,100])/1.187,
    m_z=[0,0.1],
    ph_symmetry=[0],
    num_bands=[100],
    m_eff=[1],
    p_mode=[10],
    get_wf=[False],
    sym = [True],
    A_0 = [A_0],
    B_0 = [B_0],
    C_0 = [C_0],
    C_1 = [C_1],
    C_2 = [C_2],
    M_0 = [M_0],
    M_1 = [M_1],
    M_2 = [M_2]
#    A_perp = [3],
#    A_z = [1]
)

In [ ]:
params = dict(
    k=[1e-8],
#   k=[-.1,.1],
    a=[-80],
    a_z=[-20,1],
    W=[500],
    delta=[0],
    flux=[0],
    m_z=[0],
    mu=[-100],
    m0=[0],
    m1=[0],
    D=[0],
    h_mode=[3],
#    T=[20,40],
    T=[15,100],
    ph_symmetry=[0],
    num_bands=[50],
    m_eff=[1],
    p_mode=[15],
    get_wf=[False],
    sym = [False],
)

In [ ]:
qa = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['T'],
    arg_picker=itemgetter('Es'),
    **params
)

qa.learner.strategy = "npoints"
qa.load('data/band_structures/')

In [ ]:
#qa.runner.task.result()

In [ ]:
# import adaptive
adaptive.notebook_extension()
qa.runn(
        goal=lambda learner: learner.npoints > len(params['W'])*len(params['a_z'])*100,
        executor=client
    )

In [ ]:
qa.runner.live_info()

In [ ]:
qa.save('data/band_structures/')

In [ ]:
#qa.learner.plot()

In [ ]:
from scipy.interpolate import pchip_interpolate

In [ ]:
x, Es = [], []
band_a, band_b = [], []
for pos in range(len(params['a_z'])):
    x.append(list(qa.learner.learners[pos].extra_data.keys()))
    Es.append(np.array([qa.learner.learners[pos].extra_data[key]['Es'] for key in qa.learner.learners[pos].extra_data.keys()]))
    above = [ np.min( bands[ bands > 0] ) for bands in Es[pos] ]
    below = [ np.max( bands[ bands < 0] ) for bands in Es[pos] ]
    
    above = np.array([y for _, y in sorted(zip(x[pos], above))])
    below = np.array([y for _, y in sorted(zip(x[pos], below))])
    
    x[pos] = np.sort(x[pos])
    
    band_a.append(above)
    band_b.append(below)

diffs = []

shifts = np.linspace(1,1.5,501)

for shift in shifts:
    ds0 = np.linspace(*params['T'],1000)
    ys0 = pchip_interpolate(x[0],band_a[0], ds0)
    
    ds1 = np.linspace(*params['T'],1000)/shift
    ys1 = pchip_interpolate(x[2],band_a[2], ds1)
    
    diffs.append(np.abs(np.sum(ys1 -ys0) ))
    
print(shifts[np.where( diffs == np.min(diffs))[0][0]])

In [ ]:
# For a_z = -20, shift = 1.08
# For a_z = -10, shift = 1.187

In [ ]:
plt.plot(x[0],band_a[0],"-");
plt.plot(x[0],band_b[0],"-");

plt.plot(x[2]*1.187,band_a[2],"--");
plt.plot(x[2]*1.187,band_b[2],"--");

In [ ]:
plt.style.use(['default', './paper.mplstyle'])
from matplotlib.patches import Rectangle

In [ ]:
x = []
Es = []
if len(params['T']) == 2:
    for pos in range(len(params['W'])):
        x.append(list(qa.learner.learners[pos].extra_data.keys()))
        Es.append([qa.learner.learners[pos].extra_data[key]['Es'] for key in qa.learner.learners[pos].extra_data.keys()])
elif len(params['m_z']) == 2:
    for pos in range(len(params['T'])*len(params['W'])):
        x.append(list(qa.learner.learners[pos].extra_data.keys()))
        Es.append([qa.learner.learners[pos].extra_data[key]['Es'] for key in qa.learner.learners[pos].extra_data.keys()])

if len(params['T']) == 2:
    iis = range(len(params['W']))
    fig, ax = plt.subplots(1,len(iis),figsize=(2.5*len(iis),3),layout = 'constrained')
elif len(params['m_z']) == 2:
    max_E = .07
    iis = range(len(params['T']))
    fig, ax = plt.subplots(1,len(iis),figsize=(2*len(iis),3),layout = 'constrained',sharey=True)

if params['a_z'][0] == -20:
    shift = 1.08
elif params['a_z'][0] == -10:
    shift = 1.187
    
Ts = np.array([25,40,70])
colors = ['tab:purple','tab:blue','tab:brown']
    
if params['h_mode'][0]==0:
    A_0 = funcs.param_list[params['p_mode'][0]]['A_perp']
    B_0 = funcs.param_list[params['p_mode'][0]]['A_z']
    M_0=-funcs.param_list[params['p_mode'][0]]['M_0'] 
    M_1=funcs.param_list[params['p_mode'][0]]['M_z']
#ax[0].grid();
dds = np.linspace(10,100,100)
l_c = -1#np.pi/(cmath.sqrt(.25*B_0**2/(M_1**2-C_1**2)+M_0/M_1)).imag
for ii in iis:
    for i in range(len(Es[ii])):
        ax[ii].set_yticks(np.arange(-max_E,max_E+0.05,.05), [np.round(i,2) for i in np.arange(-max_E,max_E+0.05,.05)-M_0*C_1/M_1])
        ax[ii].set_ylim(-max_E,max_E)
        if len(params['T']) == 2:
            ax[ii].plot(shift*x[ii][i]*np.ones(params['num_bands']),Es[ii][i],'ko',markersize=.75,markeredgewidth=0)            
            ax[ii].set_xlim(20,100)
            ax[ii].set_xlabel('$d~[\AA]$')
            ax[ii].set_xticks(np.arange(20,101,20))
            ax[ii].set_title('$W='+str(params['W'][ii])+"~\AA$")
            for i in range(3):
                ax[ii].plot([Ts[i]]*2,[-max_E,max_E],color=colors[i])            
        elif len(params['m_z']) == 2:
            ax[ii].plot(x[ii][i]*np.ones(params['num_bands']),Es[ii][i],'ko',markersize=.75,markeredgewidth=0)
            ax[ii].set_xlabel('$M_z~[eV]$')
            ax[ii].set_xticks(np.arange(0,0.11,0.02))
            ax[ii].set_yticks(np.arange(-.1,.1,0.025),[np.round(i,3) for i in np.arange(-.1,.1,0.025)])
            ax[ii].set_xlim(0,.1)
            ax[ii].set_ylim(-max_E,max_E)
            ax[ii].set_title('$T='+str(int(np.round(shift*params['T'][ii])))+"~\AA$")
        
#        [ax[ii].plot(dds,M_0+M_1*(n*np.pi/dds)**2,"r-") for n in [1,2]]
#        [ax[ii].plot(dds,-M_0-M_1*(n*np.pi/dds)**2,"r-") for n in [1,2]]


ax[0].set_ylabel('$E~[eV]$');
if len(params['T']) == 2:
    for ii in iis:
        [ ax[ii].add_patch(Rectangle((n*l_c,-10),l_c,20,alpha=0.7,color="r",lw=0)) for n in [1,3,5] ]
        spacing = 2*np.pi*A_0/(2*params['W'][ii]+2*100)
        ax[ii].add_patch(Rectangle((20,-spacing/2),100,spacing,alpha=0.7,color="g",lw=0))
    plt.show();
    fig.savefig('width_dependence/width_dependence_spec_d_E_sym'+str(params['sym'][0])+'_p_mode'+str(params['p_mode'][0])+
            '_m_z'+str(params['m_z'][0])+
            '_A_0='+str(np.round(A_0,3))+'_B_0='+str(np.round(B_0,3))+
            '_C_0='+str(np.round(C_0,3))+'_C_1='+str(np.round(C_1,3))+'_C_2='+str(np.round(C_2,3))+
            '_M_0='+str(np.round(M_0,3))+'_M_1='+str(np.round(M_1,3))+'_M_2='+str(np.round(M_2,3))+'.png');      
elif len(params['m_z']) == 2:
    plt.show();    
    fig.savefig('width_dependence/width_dependence_spec_E_sym'+str(params['sym'][0])+'_p_mode'+str(params['p_mode'][0])+
            '_T'+str(params['T'][0])+'_W'+str(params['W'][0])+
            '_A_0='+str(np.round(A_0,3))+'_B_0='+str(np.round(B_0,3))+
            '_C_0='+str(np.round(C_0,3))+'_C_1='+str(np.round(C_1,3))+'_C_2='+str(np.round(C_2,3))+
            '_M_0='+str(np.round(M_0,3))+'_M_1='+str(np.round(M_1,3))+'_M_2='+str(np.round(M_2,3))+'.png');

In [ ]:
2*np.pi*4.1/(2*3000)

In [ ]:
x = []
Es = []
for pos in range(len(params['T'])*len(params['W'])):
    x.append(list(qa.learner.learners[pos].extra_data.keys()))
    Es.append([ qa.learner.learners[pos].extra_data[key]['Es'][ qa.learner.learners[pos].extra_data[key]['Es'] > 0 ][:2] for key in qa.learner.learners[pos].extra_data.keys()])

max_E = .1
iis = range(len(params['T']))
jjs = range(len(params['W']))
fig, ax = plt.subplots(1,len(params['W']),figsize=(2.5*len(params['W']),2),layout = 'constrained',sharey=True)

if params['a_z'][0] == -20:
    shift = 1.08
elif params['a_z'][0] == -10:
    shift = 1.187

Ts = np.array([25,40,70])
colors = ['g','b','r']
    
if params['h_mode'][0]==0:
    A_0 = funcs.param_list[params['p_mode'][0]]['A_perp']
    B_0 = params['A_z'][0]
    M_0=-funcs.param_list[params['p_mode'][0]]['M_0'] 
    M_1=funcs.param_list[params['p_mode'][0]]['M_z']
for jj in jjs:
    for ii in np.flip(iis):
        for i in range(len(Es[ii+len(iis)*jj])):
            #ax[ii].plot(x[ii][i]*np.ones(2),Es[ii][i],'ko',markersize=.75,markeredgewidth=0)
            ax[jj].plot(x[ii+len(iis)*jj][i],Es[ii+len(iis)*jj][i][1]-Es[ii+len(iis)*jj][i][0],colors[ii]+'.',markersize=3,markeredgewidth=0)
            ax[jj].set_xlabel('$M_z~[eV]$')
            ax[jj].set_xticks(np.arange(0,0.11,0.02))
            ax[jj].set_yticks(np.arange(0,.2,0.025),[np.round(i,3) for i in np.arange(0,.2,0.025)])
            ax[jj].set_xlim(0,.1)
            ax[jj].set_ylim(0,max_E)
            ax[jj].set_title('$W='+str(params['W'][jj])+"~\AA$")            
            ax[0].set_ylabel('$E~[eV]$');
ax[0].set_ylabel('$E~[eV]$');
[ax[1].plot(-1,-1,colors[ii]+'-',markersize=3,markeredgewidth=0,label='$T='+str(int(shift*params['T'][ii]))+"~\AA$") for ii in iis]
ax[1].legend();
plt.show();
fig.savefig('width_dependence/width_dependence_spacing_E_sym'+str(params['sym'][0])+'_p_mode'+str(params['p_mode'][0])+
            '_W'+str(params['W'][0])+
            '_A_0='+str(np.round(A_0,3))+'_B_0='+str(np.round(B_0,3))+
            '_C_0='+str(np.round(C_0,3))+'_C_1='+str(np.round(C_1,3))+'_C_2='+str(np.round(C_2,3))+
            '_M_0='+str(np.round(M_0,3))+'_M_1='+str(np.round(M_1,3))+'_M_2='+str(np.round(M_2,3))+'.png');

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(5,3.5),layout = 'constrained',sharey=True)
which_T = 0

max_E = .078
b_height = .09
b_slope = -.7
b_scaling = .05
b_line_x = .018

x = []
ks, Es = [], []
for pos in range(len(params_bands['m_z'])):
    x.append(list(qa_bands.learner.learners[pos].extra_data.keys()))
    Es.append([ qa_bands.learner.learners[pos].extra_data[key]['Es'] for key in qa_bands.learner.learners[pos].extra_data.keys()])

x_M_z  = list(qa.learner.learners[which_T].extra_data.keys())
Es_M_z = [ qa.learner.learners[which_T].extra_data[key]['Es'] for key in qa.learner.learners[which_T].extra_data.keys()]
iis = range(len(params_bands['m_z']))

if params['a_z'][0] == -20:
    shift = 1.08
elif params['a_z'][0] == -10:
    shift = 1.187

colors = ['g','b','r']
markersizes = np.array([2,1,1])*2
ks = np.linspace(min(params_bands['k']),max(params_bands['k']),201)

for i in range(len(Es_M_z)):
    pts_to_plot = (Es_M_z[i] < max_E) & (Es_M_z[i] > -max_E)
    ax.plot(x_M_z[i]*np.ones(len(Es_M_z[i][pts_to_plot])),Es_M_z[i][pts_to_plot],'k.',markersize=markersizes[-1],markeredgewidth=0)   

for ii in np.flip(iis):
    min_CB = np.min( [np.min(Es[ii][i][ Es[ii][i] > 0]) for i in range(len(Es[ii]))] )
    for i in range(len(Es[ii])):
        #ax[ii].plot(x[ii][i]*np.ones(2),Es[ii][i],'ko',markersize=.75,markeredgewidth=0)
        pts_to_plot = (Es[ii][i] < b_slope*(x[ii][i]*b_scaling) + b_height ) & (Es[ii][i] > b_slope*(x[ii][i]*b_scaling) - b_height )
        ax.arrow( x=params_bands['m_z'][ii]-b_line_x, y=b_slope*-b_line_x - b_height, dx = 2*b_line_x, dy = b_slope*2*b_line_x,width=.0002)
        ax.text( x=params_bands['m_z'][ii]+b_line_x+.003, y=b_slope*b_line_x - b_height-.006, s = '$k$')        
        ax.plot(params_bands['m_z'][ii]+x[ii][i]*b_scaling*np.ones(len(Es[ii][i][pts_to_plot])),Es[ii][i][pts_to_plot],colors[ii]+'.',markersize=markersizes[ii],markeredgewidth=0)
        
ax.set_xlabel('$M_z~[eV]$')
ax.set_xticks(np.arange(0,0.11,0.02))
ax.set_yticks(np.arange(-.2,.2,0.025),[np.round(i,3) for i in np.arange(-.2,.2,0.025)])
ax.set_xlim(-.015,.1+.025)
ax.set_ylim(-b_height*1.25,b_height*1.2)
ax.set_ylabel('$E~[eV]$');
ax.set_ylabel('$E~[eV]$');
#ax.legend();
plt.show();
fig.savefig('width_dependence/M_z_and_spectrum_sym'+str(params['sym'][0])+'_p_mode'+str(params['p_mode'][0])+
            '_W'+str(params['W'][0])+
            '_A_0='+str(np.round(A_0,3))+'_B_0='+str(np.round(B_0,3))+
            '_C_0='+str(np.round(C_0,3))+'_C_1='+str(np.round(C_1,3))+'_C_2='+str(np.round(C_2,3))+
            '_M_0='+str(np.round(M_0,3))+'_M_1='+str(np.round(M_1,3))+'_M_2='+str(np.round(M_2,3))+'.png');

In [ ]:
A_0 = 3
B_0 = 3
C_0 = 0
C_1 = 0
C_2 = 0
M_0 = -0.3
M_1 = 15
M_2 = 15

In [ ]:
params_phase = dict(
    a=[10],
    a_z=[-10],
#    T=np.array([25])/1.08,#
    T=[np.round(i) for i in np.array([25,40,70])/1.187],#np.array([20,100])/1.187,
    W=[500],
    delta=[0],
    mu=[-M_0*C_1/M_1],
    m_z=[0,0.1],
    h_mode=[3],    
    p_mode=[10],
    get_delta=[False],
    k_points=[1000],
    num_bands=[50],
    sym=[False],
    A_0 = [A_0],
    B_0 = [B_0],
    C_0 = [C_0],
    C_1 = [C_1],
    C_2 = [C_2],
    M_0 = [M_0],
    M_1 = [M_1],
    M_2 = [M_2]
)

In [ ]:
qa_phase = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('mn_u'),
    **params_phase
)

qa_phase.learner.strategy = "npoints"

In [ ]:
qa_phase.load('data/band_structures/')

In [ ]:
qa_phase.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client
)

In [ ]:
qa_phase.save('data/band_structures/')

In [ ]:
params_phase2 = dict(
    a=[10],
    a_z=[-10],
#    T=np.array([25])/1.08,#
    T=[np.round(i) for i in np.array([25,40,70])/1.187],#np.array([20,100])/1.187,
    W=[2000],
    delta=[0],
    mu=[-M_0*C_1/M_1],
    m_z=[0,0.1],
    h_mode=[3],    
    p_mode=[10],
    get_delta=[False],
    k_points=[1000],
    num_bands=[50],
    sym=[False],
    A_0 = [A_0],
    B_0 = [B_0],
    C_0 = [C_0],
    C_1 = [C_1],
    C_2 = [C_2],
    M_0 = [M_0],
    M_1 = [M_1],
    M_2 = [M_2]
)

In [ ]:
qa_phase2 = tools.QuickAdptive(
    funcs.get_fig_of_merit,
    learner_bounds_names=['m_z'],
    arg_picker=itemgetter('mn_u'),
    **params_phase2
)

qa_phase2.learner.strategy = "npoints"

In [ ]:
qa_phase2.load('data/band_structures/')

In [ ]:
qa_phase2.runn(
    goal=lambda learner: learner.npoints > 300,
    executor=client2
)

In [ ]:
qa_phase2.save('data/band_structures/')

In [ ]:
modes = []
energies = []
m_zs = []

qa_plot = qa_phase2
params_p = params_phase2

nu_prop_modes, energies_prop_modes = [],[]
for pos in range(len(params_p['T'])):
    nu_prop_modes.append([qa_plot.learner.learners[pos].extra_data[key]['nu_prop_modes'] for key in qa_plot.learner.learners[pos].extra_data.keys()])
    energies_prop_modes.append([qa_plot.learner.learners[pos].extra_data[key]['energies_prop_modes'] for key in qa_plot.learner.learners[pos].extra_data.keys()])

for pos in range(len(params_p['T'])):
    modes_ = []
    energies_ = []
    m_zs_ = []
    
    for m in nu_prop_modes[pos]:
        modes_ = modes_+list(m)
    for e in energies_prop_modes[pos]:
        energies_ = energies_+list(e)
    for key in qa_plot.learner.learners[pos].extra_data.keys():
        m_zs_ = m_zs_ + [key]*len( qa_plot.learner.learners[pos].extra_data[key]['nu_prop_modes'] )
        
    modes.append(modes_)
    energies.append(energies_)
    m_zs.append(m_zs_)

In [ ]:
if params_p['a_z'][0] == -20:
    shift = 1.08
elif params_p['a_z'][0] == -10:
    shift = 1.187

max_E = .1
iis = range(len(params['T']))    
    
fig, ax = plt.subplots(1,len(params_p['T']),figsize=(2*len(params_p['T']),3),layout = 'constrained')

for ii in iis:
    ax[ii].set_xlabel('$M_z~[eV]$')
    ax[ii].set_xticks(np.arange(0,0.11,0.02))
    if params_p['C_1'][0] == 0:
        ax[ii].set_yticks(np.arange(-max_E,max_E,0.025),[np.round(i,3) for i in np.arange(-max_E,max_E,0.025)])
        ax[ii].set_ylim(-max_E,max_E)    
    else:
        ax[ii].set_yticks(np.arange(-max_E+params_p['mu'][0],max_E+params_p['mu'][0],0.025),[np.round(i,3) for i in np.arange(-max_E+params_p['mu'][0],max_E+params_p['mu'][0],0.025)])
        ax[ii].set_ylim(-max_E+params_p['mu'][0],max_E+params_p['mu'][0])
    ax[ii].set_xlim(0,.1)
    ax[ii].set_title('$T='+str(round(shift*params_p['T'][ii]))+"~\AA$")
    ax[ii].tricontourf(m_zs[ii],energies[ii],modes[ii],levels=np.arange(0,11,1))
        
#        [ax[ii].plot(dds,M_0+M_1*(n*np.pi/dds)**2,"r-") for n in [1,2]]
#        [ax[ii].plot(dds,-M_0-M_1*(n*np.pi/dds)**2,"r-") for n in [1,2]]

ax[0].set_ylabel('$E~[eV]$');
plt.show();
fig.savefig('width_dependence/width_dependence_phase_E_sym'+str(params_p['sym'][0])+'_p_mode'+str(params_p['p_mode'][0])+
            '_W'+str(params['W'][0])+
            '_A_0='+str(np.round(params_p['A_0'][0],3))+'_B_0='+str(np.round(params_p['B_0'][0],3))+
            '_C_0='+str(np.round(params_p['C_0'][0],3))+'_C_1='+str(np.round(params_p['C_1'][0],3))+'_C_2='+str(np.round(params_p['C_2'][0],3))+
            '_M_0='+str(np.round(params_p['M_0'][0],3))+'_M_1='+str(np.round(params_p['M_1'][0],3))+'_M_2='+str(np.round(params_p['M_2'][0],3))+'.png');

In [ ]:
T_analyze = 50
shift = 2
wf_nu = 4

which_m_zs = [0,1]
index_T = []
indexes = []

for i in np.arange(len(which_m_zs)):
    df = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()
    x = df['x']
    for j in np.arange(len(x)):
        if T_analyze-1 < x[j] < T_analyze+1:
            break  
    index_T.append(j)
    Es = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['y'].values
    #These have to be set by hand
    if T_analyze == 30 and not params['sym'][0]:
        if i == 0 or i == 1:
            indexes.append( np.where(Es[index_T[i]] > -0.0)[0][0]+shift )
        else:
            indexes.append( np.where(Es[index_T[i]] > -0.02)[0][0]+shift )
    if T_analyze == 80 and not params['sym'][0]:
        if i == 2:
            indexes.append( np.where(Es[index_T[i]] > 0)[0][0]+shift+1 )
        else:
            indexes.append( np.where(Es[index_T[i]] > 0.02)[0][0]+shift )   
    if params['sym'][0]:
        indexes.append( np.where(Es[index_T[i]] > 0)[0][0]+shift )

In [ ]:
fig, ax = plt.subplots(1,len(index_T),figsize=(len(index_T)*3,3))
plt.style.use(['default', './paper.mplstyle'])

for i in np.arange(len(which_m_zs)):
    T = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['x'].values
    Es = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['y'].values
    for j in np.arange(len(Es)):
        ax[i].plot(T[j]*np.ones(len(Es[j])), Es[j], 'k.',markersize=2)
        
    ax[i].plot(T[index_T[i]]*np.ones(wf_nu),Es[index_T[i]][indexes[i]:indexes[i]+wf_nu], 'r.',markersize=10)
    ax[i].set_xlim(T_analyze-10,T_analyze+10)
    if not params['sym'][0]:
        ax[i].set_ylim(-0.04,0.11);
    else:
        ax[i].set_ylim(-0.12,0.12);
    ax[i].set_xlabel('$d~[\AA]$'); 
    ax[i].set_title('$d='+str(T_analyze)+'~\AA,~M_z='+str(which_m_zs[i]/100)+'~eV$')
    
ax[0].set_ylabel('$E~[eV]$');

plt.savefig('wf_analysis_sym'+str(params['sym'][0])+'_d='+str(T_analyze)+'_wfs'+str(wf_nu)+'_spectrum.png')
plt.show()

In [ ]:
import kwant

In [ ]:
fig, ax = plt.subplots(wf_nu,len(index_T),figsize=(len(index_T)*4,wf_nu))
spin = 'z'

for i in np.arange(len(which_m_zs)):
    extra_data = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['extra_data']
    T = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['x'][index_T[i]]
    Es = qa.runner.learner.learners[which_m_zs[i]].to_dataframe()['y'][index_T[i]]    
    wfs = extra_data[index_T[i]]['wfs']
    a   = extra_data[index_T[i]]['a']
    a_z = extra_data[index_T[i]]['a_z']

    lead = systems.make_lead(
    a=a, a_z=a_z, W=500, T=T,
    ph_symmetry=False,
    vector_potential="[0, - B_x * (z - {}), 0]".format(T),
    subst={'Delta': "Delta_lead(delta, x, y, z)"},
    ham_type='3D'
    )
    if spin == 'z':
        rho = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_z, systems.sigma_0))
    else:
        rho = kwant.operator.Density(lead.finalized(), np.kron(systems.sigma_0, systems.sigma_0))
    for j in np.arange(indexes[i],indexes[i]+wf_nu):
        density = rho(wfs[:,j])
        ax[indexes[i]+wf_nu-1-j][i].contourf(np.arange(0,500/a+1),np.arange(0,T/a_z+1),density.reshape(int(500/a)+1,int(T/a_z)+1).T,vmin=-0.005,vmax=0.005,levels=100,cmap='rainbow')
        ax[indexes[i]+wf_nu-1-j][0].set_ylabel('$z~[a_z]$');
    ax[3][i].set_xlabel('$y~[a]$');
    ax[0][i].set_title('$\langle\Psi|\sigma_'+spin+'|\Psi\\rangle,~d='+str(T_analyze)+'~\AA,~M_z='+str(which_m_zs[i]/100)+'~eV$')

plt.savefig('wf_analysis_sym'+str(params['sym'][0])+'_d='+str(T_analyze)+'_wfs'+str(wf_nu)+'_wfs_sigma_'+spin+'.png')
plt.show()

In [ ]:
Ts = [qa.learner.learners[i].to_dataframe()["x"] for i in range(len(params['a_z']))]
Es = [qa.learner.learners[i].to_dataframe()["y"] for i in range(len(params['a_z']))]
#gaps = [[min(Es[j][i][Es[j][i] > 0]) - max(Es[j][i][Es[j][i] < 0]) for i in np.arange(len(Es[j]))] for j in range(len(params['a_z'])) ]
#E_u = [[min(Es[j][i][Es[j][i] > min(Es[j][i][Es[j][i] > 0])]) - min(Es[j][i][Es[j][i] > 0]) for i in np.arange(len(Es[j]))] for j in range(len(params['a_z'])) ]

In [ ]:
from scipy.interpolate import PchipInterpolator

In [ ]:
thres = 0.0125*np.ones(len(Es[0].values))#[0.05-0.02*(i-17)/80 for i in range(len(Es[0].values))]

lower_10=np.array([np.max(Es[0].values[i][Es[0].values[i]<thres[i]]) for i in range(len(Es[0].values))])
upper_10=np.array([np.min(Es[0].values[i][Es[0].values[i]>thres[i]]) for i in range(len(Es[0].values))])
upper_102=np.array([np.min(Es[0].values[i][Es[0].values[i]>np.min(Es[0].values[i][Es[0].values[i]>thres[i]])]) for i in range(len(Es[0].values))])

#thres = [0.05-0.02*(i-20)/80 for i in range(len(Es[1].values))]
thres = 0.0125*np.ones(len(Es[1].values))

lower_100=np.array([np.max(Es[1].values[i][Es[1].values[i]<thres[i]]) for i in range(len(Es[1].values))])
upper_100=np.array([np.min(Es[1].values[i][Es[1].values[i]>thres[i]]) for i in range(len(Es[1].values))])
upper_1002=np.array([np.min(Es[1].values[i][Es[1].values[i]>np.min(Es[1].values[i][Es[1].values[i]>thres[i]])]) for i in range(len(Es[1].values))])

In [ ]:
f_upper_102 = PchipInterpolator(Ts[0], upper_102)
f_upper_10 = PchipInterpolator(Ts[0], upper_10)
f_lower_10 = PchipInterpolator(Ts[0], lower_10)

f_upper_1002 = PchipInterpolator(Ts[1], upper_1002)
f_upper_100 = PchipInterpolator(Ts[1], upper_100)
f_lower_100 = PchipInterpolator(Ts[1], lower_100)

Ts_check = np.linspace(25,100,1000)
ds = np.linspace(1,2,401)
diffs = [np.sum((f_upper_10(Ts_check/d) - f_lower_10(Ts_check/d) - f_upper_100(Ts_check) + f_lower_100(Ts_check))**2) for d in ds]

shift = ds[ np.where( diffs == np.min(diffs))[0][0] ]
print(shift)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,4),sharey=True)

[ax[0].plot(Ts[0][i]*np.ones(len(Es[0][i])),Es[0][i],'k.') for i in range(len(Es[0]))];
ax[0].set_ylim(0,0.15);
#ax[0].plot(Ts[0],lower_10,'g-',lw=3)
#ax[0].plot(Ts[0],upper_10,'r-',lw=3)

[ax[1].plot(Ts[1][i]*np.ones(len(Es[1][i])),Es[1][i],'k.') for i in range(len(Es[1]))];
ax[1].set_ylim(0,0.15);
ax[1].plot(Ts[1],lower_100,'g-',lw=3)
ax[1].plot(Ts[1],upper_100,'r-',lw=3)

ax[2].plot(Ts[1],lower_100,'g-',lw=3)
ax[2].plot(Ts[1],upper_100,'r-',lw=3)
#ax[2].plot(Ts[1],upper_1002,'b-',lw=3)

ax[2].plot(Ts[0]*shift,lower_10,'k--')
ax[2].plot(Ts[0]*shift,upper_10,'k--')
#ax[2].plot(Ts[0]*shift,upper_102,'k--')

ax[0].set_xlim(10,100)
ax[2].set_ylim(-.1,0.1)

fig.suptitle("Spectrum a_z="+", ".join([str(i) for i in params['a_z']])+", shift="+str(shift))
plt.savefig("spectrum_a_z="+"_".join([str(i) for i in params['a_z']])+".png")

In [ ]:
ds = qa.learner.learners[i].to_dataframe()["x"].values
B_0s = qa.learner.learners[i].to_dataframe()["y"].values
gaps = qa.learner.learners[i].to_dataframe()["z"].values

if params['h_mode'][0]==0:
    A_0 = funcs.param_list[params['p_mode'][0]]['A_perp']
    B_0 = funcs.param_list[params['p_mode'][0]]['A_z']
    M_0=-funcs.param_list[params['p_mode'][0]]['M_0']
    M_1=funcs.param_list[params['p_mode'][0]]['M_z']
else:
    A_0 = funcs.param_list[params['p_mode'][0]]['A_0']
    B_0 = funcs.param_list[params['p_mode'][0]]['B_0']
    M_0=funcs.param_list[params['p_mode'][0]]['M_0']
    M_1=funcs.param_list[params['p_mode'][0]]['M_1']

In [ ]:
plt.tricontourf(B_0s, ds*1.187, gaps,levels=np.linspace(0,0.05,100) );
plt.plot([np.sqrt(-2*M_0*M_1)]*2, [10,100] );
plt.colorbar();
plt.show()

In [ ]:
params = dict(  
    a=[500/30],
    a_z=[8],
    T=[80],    
    W=[500],
    delta=[0],
    flux=[0],
    m_z=[0.02],    
    mu=[0],
    m0=[0],
    m1=[0],
    D=[0],    
    h_mode=[0],
    ph_symmetry=[0],    
    p_mode=[0],
    get_wf=[True]
)

delta = 0.001
npoints_1 = 300
npoints_2 = 300

In [ ]:
qa = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params,
    k=[-.5,.5],            
    num_bands=[30],    
    m_eff=[1],    
)

qa.learner.strategy = "npoints"

In [ ]:
#qa.runner.task.result()

In [ ]:
qa.load('./data/band_structures');

In [ ]:
import adaptive
adaptive.notebook_extension()

qa.runn(
    goal=lambda learner: learner.npoints > npoints_2*3,
    executor=client
)

In [ ]:
executor = client.executor()
job = executor.submit(funcs.get_fig_of_merit, a=params['a'][0], a_z=params['a_z'][0],
                                        T=params['T'][0], W=params['W'][0], delta=params['delta'][0], 
                                        m_z=params['m_z'][0], mu=params['mu'][0], 
                                        h_mode=params['h_mode'][0], p_mode=params['p_mode'][0])

In [ ]:
result_gaps = job.result()

In [ ]:
params_u = params.copy()
params_l = params.copy()
params_u_delta = params.copy()
params_l_delta = params.copy()
params_u.update(mu=[result_gaps["mu_u"]],ph_symmetry=[1])
params_l.update(mu=[result_gaps["mu_l"]],ph_symmetry=[1])
params_u_delta.update(mu=[result_gaps["mu_u"]],ph_symmetry=[1],delta=[delta])
params_l_delta.update(mu=[result_gaps["mu_l"]],ph_symmetry=[1],delta=[delta])

In [ ]:
lim_u = [-0.1,0.1]
lim_l = [-0.5,0.5]

qa_u = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params_u,
    k=lim_u,            
    num_bands=[6],    
    m_eff=[1],    
)
qa_l = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params_l,
    k=lim_l,            
    num_bands=[6],    
    m_eff=[1],    
)

qa_u_delta = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params_u_delta,
    k=lim_u,            
    num_bands=[6],    
    m_eff=[1],    
)
qa_l_delta = tools.QuickAdptive(
    funcs.calculate_lead_spectrum,
    learner_bounds_names=['k'],
    arg_picker=itemgetter('Es'),
    **params_l_delta,
    k=lim_l,            
    num_bands=[6],    
    m_eff=[1],    
)

In [ ]:
qa.load('./data/band_structures'); qa_u.load('./data/band_structures'); qa_l.load('./data/band_structures');
qa_u_delta.load('./data/band_structures'); qa_l_delta.load('./data/band_structures');

In [ ]:
qa_u.runn(
    goal=lambda learner: learner.npoints > npoints_1,
    executor=client
)
qa_l.runn(
    goal=lambda learner: learner.npoints > npoints_1,
    executor=client
)
qa_u_delta.runn(
    goal=lambda learner: learner.npoints > npoints_2,
    executor=client
)
qa_l_delta.runn(
    goal=lambda learner: learner.npoints > npoints_2,
    executor=client
)

In [ ]:
qa.save('./data/band_structures'); qa_u.save('./data/band_structures'); qa_l.save('./data/band_structures');
qa_u_delta.save('./data/band_structures'); qa_l_delta.save('./data/band_structures');

In [ ]:
import plot
from itertools import product
import pickle
import importlib
importlib.reload(plot)
import numpy as np
import matplotlib as mpl
from matplotlib import colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.pyplot as plt
import matplotlib.patheffects as patheffects
from matplotlib import rcParams
from cycler import cycler
from matplotlib.patches import Rectangle
mpl.rcParams.update(mpl.rcParamsDefault)
textwidth = 5.45776
golden_ratio = 0.6180339887498949
plt.style.use(['default', './paper.mplstyle'])

In [ ]:
fig = plt.figure(
    figsize=(textwidth, textwidth/2),
)

gs0 = fig.add_gridspec(1, 2, left=0.08, right=1.048, top=0.92, bottom=0.16, wspace=0.25, width_ratios=[1/2, 1/2])

bottom = result_gaps["mu_l"]-result_gaps["E_l"]/2
height = result_gaps["E_l"]+2*result_gaps["E_gap"]+result_gaps["E_u"]

ax1 = fig.add_subplot(gs0[0])
ax1.add_patch(Rectangle((-.5,bottom),1,result_gaps["E_l"],alpha=0.5,color="g"))
ax1.add_patch(Rectangle((-.5,result_gaps["mu_gap"]-result_gaps["E_gap"]/2),1,result_gaps["E_gap"],alpha=0.5,color="b"))
ax1.add_patch(Rectangle((-.5,result_gaps["mu_u"]-result_gaps["E_u"]/2),1,result_gaps["E_u"],alpha=0.5,color="r"))

line = plot.spectrum(
    folder="./data/band_structures",
    ax=ax1,
    delta_zero='only',
    gap_line=False,
    E_scale=1,
    a=params['a'][0], 
    a_z=params['a_z'][0], 
    T=params['T'][0], W=params['W'][0], delta=params['delta'][0], 
    flux=params['flux'][0], m_z=params['m_z'][0], mu=params['mu'][0],
    m0=params['m0'][0], m1=params['m1'][0], D=params['D'][0], 
    h_mode=params['h_mode'][0], ph_symmetry=params['ph_symmetry'][0], p_mode=params['p_mode'][0],
    get_wf=params['get_wf'][0],num_bands=30,    
    m_eff=1,
)

ax1.set_xlabel('$k$ [1/nm]')
ax1.set_ylabel('$E$ [eV]')
ax1.set_xlim(-0.5, 0.5)
ax1.set_ylim(bottom-0.2*height, bottom+1.2*height)
ax1.axhline(y=result_gaps["mu_u"], linestyle='--', c='black',lw=.5)
ax1.axhline(y=result_gaps["mu_l"], linestyle='--', c='black',lw=.5)

gs1 = gs0[1].subgridspec(2, 1, wspace=0, hspace=0)
#axs = gs1.subplots(sharex='col', sharey='row')
axs = gs1.subplots()
mus = np.array([result_gaps["mu_u"], result_gaps["mu_l"]])
for ax, mu in zip(axs.flat,  mus):
    lcs, line = plot.spectrum(
        folder="./data/band_structures",
        ax=ax,
        delta_zero=True,
        a=params['a'][0], 
        a_z=params['a_z'][0], 
        T=params['T'][0], W=params['W'][0], delta=delta, 
        flux=params['flux'][0], m_z=params['m_z'][0], mu=mu,
        m0=params['m0'][0], m1=params['m1'][0], D=params['D'][0], 
        h_mode=params['h_mode'][0], ph_symmetry=1, p_mode=params['p_mode'][0], get_wf=params['get_wf'][0], 
        num_bands=6,    
        m_eff=1,  
    )

lim_k = np.array([-0.5,0.5])

axs[0].set_xlim(lim_k/10)
axs[1].set_xlim(lim_k/3)

axs[0].set_ylim([-1.1,1.1])
axs[1].set_ylim([-1.1,1.1])

    
axs[0].plot(0.5, 0.9, marker='s', markersize=5, c='r', alpha=.5, transform=axs[0].transAxes)
axs[1].plot(0.5, 0.9, marker='s', markersize=5, c='g', alpha=.5, transform=axs[1].transAxes)

axs[1].set_xlabel('$k$ [1/nm]')
axs[0].set_ylabel('$E/|\Delta|$')
axs[1].set_ylabel('$E/|\Delta|$')


ax1.text(
    -1.9, 1.1, 
    '(a)',
    transform=axs[0].transAxes
)

axs[0].text(
    -0.3, 1.1, 
    '(b)', 
    transform=axs[0].transAxes
)

#cbar = fig.colorbar(lcs[0], ax=axs.ravel())
cbar = fig.colorbar(lcs[0], ax=axs.ravel(), aspect=30, pad=0.01)
cbar.ax.set_yticklabels(['h'] + (len(cbar.get_ticks())-2)*[''] + ['p'])
cbar.ax.tick_params(direction='inout', which='both')
    
plt.show();

In [ ]:
#fig.savefig('./figures/figure_100_0.02_Bi2Se3old.pdf')
#fig.savefig('./figures/figure_100_0.02_Bi2Se3old.png', dpi=500)